In [26]:
import zipfile, re, time
import psycopg2 as p
import psycopg2.extras as e
from datetime import datetime

In [7]:
start_time = time.time()

In [2]:
password = 'Evergreen1'

conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
                       host = 'localhost', port = 5433)

cur = conn.cursor()

# Update_review for NMC and PFDs

In [6]:
# #For DOT's missing and for MCS-150 dates over thresholds
# #Greater than 20 total vehicles
# #password = 'Evergreen1'

# #conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
# #                       host = 'localhost', port = 5433)
# #cur = conn.cursor()

# # dd/mm/YY H:M:S
# #dt_string = datetime.now().strftime("%Y_%m_%d %H_%M_%S")


# #New query as of 1/3/2020
# query = """select *

# from ( 
#     select 
#         f.fleet_id, f.product_code, f.company_name, f.website, f.year_established, f.revenue, f.mcn, f.usdot, f.scac, f.company_type_code, 
#         f.fleet_type_id, f.safety_rating_type_id, f.onsite_maintenance, f.tl_ltl, f.trucks_owned, f.trucks_leased, 
#         f.tractors_owned, f.tractors_leased, f.trailers_owned, f.trailers_leased, f.gvw_class_12, f.gvw_class_345, f.gvw_class_6, f.gvw_class_7,
#         f.gvw_class_8, f.sic_code, f.insurance_info_flag, f.update_date, f.active_flag, f.calculated_revenue, f.haz_trailers_owned, f.haz_trailers_leased,
#         f.haz_trucks_owned, f.haz_trucks_leased, f.vehicle_miles, f.drivers, f.cdl_drivers, f.average_miles, f.gallonmin, f.gallonmax, 
#         f.dot_hmind, f.total_trucks, f.total_tractors, f.total_trailers, f.total_vehicles, f.total_vehicles_owned, f.total_vehicles_leased,
#         f.last_called_date, f.note as f_note, ct.description as company_type_name, ct.type as company_type, ft.name as fleet_type, 
#         case when pn1.phone_type_id = 1 then pn1.phone_number else null end as phone_number, case when pn2.phone_type_id = 2 then pn2.phone_number else null end as tf_number,
#         case when pn3.phone_type_id = 3 then pn3.phone_number else null end as fax_number, l1.address1 as phy_address1, l1.address2 as phy_address2, l1.address3 as phy_address3,
#         l1.city as phy_city, l1.county_fips as phy_county_fips, l1.state_code as phy_state_code, l1.zip_code as phy_zip_code, l1.country_code as phy_country_code,
#         l1.latitude as phy_latitude, l1.longitude as phy_longitude, l2.address1 as mai_address1, l2.address2 as mai_address2, l2.address3 as mai_address3,
#         l2.city as mai_city, l2.county_fips as mai_county_fips, l2.state_code as mai_state_code, l2.zip_code as mai_zip_code, l2.country_code as mai_country_code,
#         l2.latitude as mai_latitude, l2.longitude as mai_longitude, '' as commodities, '' as trailer_types, f.create_date, m.mcmis_id, m.act_stat, m.carship,
#         m.census_num, m.name, m.name_dba, m.dbnum, m.phy_natn, m.reg, m.phy_str, m.phy_city, m.phy_cnty, m.phy_st, m.phy_zip, m.undeliv_phy, 
#         m.tel_num, m.fax_num, m.mai_natn, m.mai_str, m.mai_city, m.mai_cnty, m.mai_st, m.mai_zip, m.undeliv_mai, m.iccdocket1, m.icc1, m.class, 
#         m.classdef, m.crrinter, m.crrhmintra, m.crrintra, m.shpinter, m.shpintra, m.genfreight, m.household, m.metalsheet, m.motorveh,
#         m.drivetow, m.logpole, m.bldgmat, m.mobilehome, m.machlrg, m.produce, m.oilfield, m.liqgas, m.intermodal, m.passengers, m.livestock,
#         m.grainfeed, m.coalcoke, m.meat, m.garbage, m.usmail, m.chem, m.drybulk, m.coldfood, m.beverages, m.paperprod, m.utility,
#         m.farmsupp, m.construct, m.waterwell, m.cargoothr, m.othercargo, m.hm_ind, m.owntruck, m.owntract, m.owntrail, m.trmtruck, 
#         m.trmtract, m.trmtrail, m.trptruck, m.trptract, m.trptrail, m.tot_trucks, m.tot_pwr, m.interlt100, m.intergt100, m.inter_drs, m.intralt100,
#         m.intragt100, m.intra_drs, m.avg_tld, m.tot_drs, m.cdl_drs, m.mlg150, m.miletot, m.mcs150_date, m.email_address, m.company_rep1, m.company_rep2,

#         case 
#             when (f.total_vehicles >= 20 AND m.mcs150_date < now() - interval '30 months' and f.product_code = 'NMC' and f.active_flag = 'Y' or 
#             (f.total_vehicles >= 20 and m.mcs150_date < now() - interval '36 months' and f.product_code = 'PFD' and f.active_flag = 'Y')) then 'Exceeds MCS-150 Threshold'
#         end as mcmis_review_reason,
#         ' ' as note

#     from fleetseek.fleet as f
#     join mcmis as m
#     on f.usdot = m.census_num
#     join fleetseek.fleet_primary as fp
#     on f.fleet_id = fp.fleet_id
#     left join fleetseek.phone_number as pn1
#     on fp.primary_phone_id = pn1.phone_number_id
#     left join fleetseek.phone_number as pn2
#     on fp.primary_toll_free_id = pn2.phone_number_id
#     left join fleetseek.phone_number as pn3
#     on fp.primary_fax_id = pn3.phone_number_id
#     left join fleetseek.location as l1
#     on fp.primary_physical_location_id = l1.location_id
#     left join fleetseek.location as l2 
#     on fp.primary_mailing_location_id = l2.location_id
#     left join fleetseek.company_type as ct
#     on f.company_type_code = ct.company_type_code
#     left join fleetseek.fleet_type as ft
#     on f.fleet_type_id = ft.fleet_type_id
#     ) as a

# where 
#     a.usdot != 0 and  
#     a.mcmis_review_reason is not null and
#     a.active_flag = 'Y'



# union all



# select *
# from (
#     select 
#         f.fleet_id, f.product_code, f.company_name, f.website, f.year_established, f.revenue, f.mcn, f.usdot, f.scac, f.company_type_code, 
#         f.fleet_type_id, f.safety_rating_type_id, f.onsite_maintenance, f.tl_ltl, f.trucks_owned, f.trucks_leased, 
#         f.tractors_owned, f.tractors_leased, f.trailers_owned, f.trailers_leased, f.gvw_class_12, f.gvw_class_345, f.gvw_class_6, f.gvw_class_7,
#         f.gvw_class_8, f.sic_code, f.insurance_info_flag, f.update_date, f.active_flag, f.calculated_revenue, f.haz_trailers_owned, f.haz_trailers_leased,
#         f.haz_trucks_owned, f.haz_trucks_leased, f.vehicle_miles, f.drivers, f.cdl_drivers, f.average_miles, f.gallonmin, f.gallonmax, 
#         f.dot_hmind, f.total_trucks, f.total_tractors, f.total_trailers, f.total_vehicles, f.total_vehicles_owned, f.total_vehicles_leased,
#         f.last_called_date, f.note as f_note, ct.description as company_type_name, ct.type as company_type, ft.name as fleet_type, 
#         case when pn1.phone_type_id = 1 then pn1.phone_number else null end as phone_number, case when pn2.phone_type_id = 2 then pn2.phone_number else null end as tf_number,
#         case when pn3.phone_type_id = 3 then pn3.phone_number else null end as fax_number, l1.address1 as phy_address1, l1.address2 as phy_address2, l1.address3 as phy_address3,
#         l1.city as phy_city, l1.county_fips as phy_county_fips, l1.state_code as phy_state_code, l1.zip_code as phy_zip_code, l1.country_code as phy_country_code,
#         l1.latitude as phy_latitude, l1.longitude as phy_longitude, l2.address1 as mai_address1, l2.address2 as mai_address2, l2.address3 as mai_address3,
#         l2.city as mai_city, l2.county_fips as mai_county_fips, l2.state_code as mai_state_code, l2.zip_code as mai_zip_code, l2.country_code as mai_country_code,
#         l2.latitude as mai_latitude, l2.longitude as mai_longitude, '' as commodities, '' as trailer_types, f.create_date, m.mcmis_id, m.act_stat, m.carship,
#         m.census_num, m.name, m.name_dba, m.dbnum, m.phy_natn, m.reg, m.phy_str, m.phy_city, m.phy_cnty, m.phy_st, m.phy_zip, m.undeliv_phy, 
#         m.tel_num, m.fax_num, m.mai_natn, m.mai_str, m.mai_city, m.mai_cnty, m.mai_st, m.mai_zip, m.undeliv_mai, m.iccdocket1, m.icc1, m.class, 
#         m.classdef, m.crrinter, m.crrhmintra, m.crrintra, m.shpinter, m.shpintra, m.genfreight, m.household, m.metalsheet, m.motorveh,
#         m.drivetow, m.logpole, m.bldgmat, m.mobilehome, m.machlrg, m.produce, m.oilfield, m.liqgas, m.intermodal, m.passengers, m.livestock,
#         m.grainfeed, m.coalcoke, m.meat, m.garbage, m.usmail, m.chem, m.drybulk, m.coldfood, m.beverages, m.paperprod, m.utility,
#         m.farmsupp, m.construct, m.waterwell, m.cargoothr, m.othercargo, m.hm_ind, m.owntruck, m.owntract, m.owntrail, m.trmtruck, 
#         m.trmtract, m.trmtrail, m.trptruck, m.trptract, m.trptrail, m.tot_trucks, m.tot_pwr, m.interlt100, m.intergt100, m.inter_drs, m.intralt100,
#         m.intragt100, m.intra_drs, m.avg_tld, m.tot_drs, m.cdl_drs, m.mlg150, m.miletot, m.mcs150_date, m.email_address, m.company_rep1, m.company_rep2,

#         case 
#             when (m.census_num is null and f.total_vehicles >= 20 and f.product_code in ('NMC', 'PFD') and f.active_flag = 'Y') then 'Inactive DOT' end as mcmis_review_reason,
#         ' ' as note

#     from fleetseek.fleet as f
#     join fleetseek.fleet_primary as fp
#     on f.fleet_id = fp.fleet_id
#     left join fleetseek.phone_number as pn1
#     on fp.primary_phone_id = pn1.phone_number_id
#     left join fleetseek.phone_number as pn2
#     on fp.primary_toll_free_id = pn2.phone_number_id
#     left join fleetseek.phone_number as pn3
#     on fp.primary_fax_id = pn3.phone_number_id
#     left join fleetseek.location as l1
#     on fp.primary_physical_location_id = l1.location_id
#     left join fleetseek.location as l2 
#     on fp.primary_mailing_location_id = l2.location_id
#     left join fleetseek.company_type as ct
#     on f.company_type_code = ct.company_type_code
#     left join fleetseek.fleet_type as ft
#     on f.fleet_type_id = ft.fleet_type_id
#     left join mcmis as m
#     on f.usdot = m.census_num
#     where m.census_num is null
#     ) as b
# where b.usdot != 0 and
# b.mcmis_review_reason is not null and
# b.active_flag = 'Y'
# """

# #outputquery = "COPY ({0}) TO STDOUT WITH CSV HEADER".format(query)


# #with open(r'\\informaplc.sharepoint.com@SSL\DavWWWRoot\sites\GAATLFS1\Shared Documents\FleetSeek Atlanta\ETL_updates\Update_output\Update_output' + dt_string + '.csv', 'w') as f:
# #    cur.copy_expert(outputquery, f)
    
    
    

In [14]:
import requests, json, time
import psycopg2 as p
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime

In [31]:
start_time = time.time()

In [28]:
password = 'Evergreen1'

conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
                       host = 'localhost', port = 5433)
cur = conn.cursor()

## Deactivations based on MCS150 date

In [29]:
# FH fleets with 20+ vehicles that have MCS150 dates that are too old
cur.execute("""
insert into fleetseek.update_output 
select
    '',
    m.icc1,
    m.act_stat,
    m.carship,
    m.census_num,
    m.dbnum,
    m.rating,
    m.chngdate,
    m.mcs150_date,
    m.class,
    m.classdef,
    m.name,
    m.name_dba,
    m.crrinter,
    m.crrhmintra,
    m.crrintra,
    m.shpinter,
    m.shpintra,
    m.org,
    m.genfreight,
    m.household,
    m.metalsheet,
    m.motorveh,
    m.drivetow,
    m.logpole,
    m.bldgmat,
    m.mobilehome,
    m.machlrg,
    m.produce,
    m.oilfield,
    m.liqgas,
    m.intermodal,
    m.passengers,
    m.livestock,
    m.grainfeed,
    m.coalcoke,
    m.meat,
    m.garbage,
    m.usmail,
    m.chem,
    m.drybulk,
    m.coldfood,
    m.beverages,
    m.paperprod,
    m.utility,
    m.farmsupp,
    m.construct,
    m.waterwell,
    m.cargoothr,
    m.othercargo,
    m.hm_ind,
    m.owntruck,
    m.owntract,
    m.owntrail,
    m.trmtruck,
    m.trmtract,
    m.trmtrail,
    m.trptruck,
    m.trptract,
    m.trptrail,
    m.tot_trucks,
    m.tot_buses,
    m.tot_pwr,
    m.phy_str,
    ubci.phy_address,
    m.phy_city,
    ubci.phy_city,
    m.phy_st,
    ubci.phy_state,
    m.phy_zip,
    ubci.phy_zip_code,
    m.phy_natn,
    m.phy_cnty,
    ubci.phy_countyname,
    ubci.phy_latitude,
    ubci.phy_longitude,
    m.undeliv_phy,
    m.mai_str,
    ubci.mai_address,
    m.mai_city,
    ubci.mai_city,
    m.mai_st,
    ubci.mai_state,
    m.mai_zip,
    ubci.mai_zip_code,
    m.mai_natn,
    m.mai_cnty,
    ubci.mai_countyname,
    ubci.mai_latitude,
    ubci.mai_longitude,
    m.undeliv_mai,
    m.tel_num,
    m.fax_num,
    m.email_address,
    f.average_miles,
    m.tot_drs,
    m.cdl_drs,
    m.mcs150mileageyear,
    m.mlg150,
    m.ratedate,
    m.adddate,
    m.acc_rate,
    m.company_rep1,
    m.company_rep2,
    f.fleet_id,
    f.usdot,
    f.mcn,
    f.sic_code,
    f.update_date,
    f.active_flag,
    sub_a.commodities,
    f.company_type_code,
    f.product_code,
    f.company_name,

    li.phy_address1,
    li.phy_address2,
    li.phy_address3,
    li.phy_state,
    li.phy_city,
    li.phy_country,
    li.phy_zip,
    li.phy_countyfips,
    li.phy_latitude,
    li.phy_longitude,
    li.phy_location_id,
    li.mai_address1,
    li.mai_address2,
    li.mai_address3,
    li.mai_state,
    li.mai_city,
    li.mai_country,
    li.mai_zip,
    li.mai_countyfips,
    li.mai_latitude,
    li.mai_longitude,
    li.mai_location_id,

    c.first_name,
    c.middle_name,
    c.last_name,
    c.suffix,
    c.original_job_title,
    c.contact_id,
    sub_b.first_name, 
    sub_b.middle_name,
    sub_b.last_name,
    sub_b.suffix,
    sub_b.original_job_title,
    sub_b.contact_id,
    sub_c.phone_number,
    sub_c.phone_number_id,
    sub_c.tf_number,
    sub_c.tf_number_id,
    sub_c.fax_number,
    sub_c.fax_number_id,
    e.email_address,
    e.email_id,
    f.trucks_owned,
    f.trucks_leased,
    f.haz_trucks_owned,
    f.haz_trucks_leased,
    f.tractors_owned,
    f.tractors_leased,
    f.total_vehicles,
    f.trailers_owned,
    f.trailers_leased,
    f.haz_trailers_owned,
    f.haz_trailers_leased,
    sub_d.trailer_type_id,
    f.drivers,
    f.cdl_drivers,
    f.gvw_class_12,
    f.gvw_class_345,
    f.gvw_class_6,
    f.gvw_class_7,
    f.gvw_class_8,
    f.revenue,
    f.website,
    'Exceeds MCS-150 Threshold',
    ''
from fleetseek.fleet as f
join mcmis as m
on f.usdot = m.census_num
left join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
left join fleetseek.updates_bn_coder_input as ubci
on f.fleet_id = ubci.fs_fleet_id
left join (
    select 
        fp.fleet_id,
        l1.address1 as phy_address1,
        l1.address2 as phy_address2,
        l1.address3 as phy_address3,
        l1.state_code as phy_state,
        l1.city as phy_city,
        l1.country_code as phy_country,
        l1.zip_code as phy_zip,
        l1.county_fips as phy_countyfips,
        l1.latitude as phy_latitude,
        l1.longitude as phy_longitude,
        l1.location_id as phy_location_id,
        l2.address1 as mai_address1,
        l2.address2 as mai_address2,
        l2.address3 as mai_address3,
        l2.state_code as mai_state,
        l2.city as mai_city,
        l2.country_code as mai_country,
        l2.zip_code as mai_zip,
        l2.county_fips as mai_countyfips,
        l2.latitude as mai_latitude,
        l2.longitude as mai_longitude,
        l2.location_id as mai_location_id

    from fleetseek.location as l1
    join fleetseek.fleet_primary as fp
    on l1.location_id = fp.primary_physical_location_id
    join fleetseek.location as l2
    on fp.primary_mailing_location_id = l2.location_id

) as li
on f.fleet_id = li.fleet_id
left join fleetseek.contact as c
on fp.primary_contact_id = c.contact_id
left join(
    select 
        min(c.contact_id), 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id
    from fleetseek.fleet_contact as fc
    join fleetseek.fleet as f
    on fc.fleet_id = f.fleet_id
    join fleetseek.contact as c
    on fc.contact_id = c.contact_id
    where 
        sort_order = 2 and
        c.active = true 
    group by 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id

) as sub_b
on fp.fleet_id = sub_b.fleet_id
left join (
    select 
        fleet_id, 
        string_agg(commodity_id::varchar, ', ') as commodities
    from fleetseek.fleet_commodity
    group by fleet_id
) as sub_a
on f.fleet_id = sub_a.fleet_id
left join (
    select 
        pn.phone_number as phone_number, 
        pn.phone_number_id as phone_number_id, 
        pn2.phone_number as fax_number, 
        pn2.phone_number_id as fax_number_id,
        pn3.phone_number as tf_number, 
        pn3.phone_number_id as tf_number_id,
        fp.fleet_id
    from fleetseek.fleet_primary as fp
    left join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id 
    left join fleetseek.phone_number as pn2
    on fp.primary_fax_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_toll_free_id = pn3.phone_number_id
) as sub_c
on f.fleet_id = sub_c.fleet_id
left join fleetseek.contact_email as ce
on sub_b.contact_id = ce.contact_id
left join fleetseek.email as e
on ce.email_id = e.email_id
left join (
    select 
        fleet_id, 
        string_agg(trailer_type_id::varchar, ', ') as trailer_type_id
    from fleetseek.fleet_trailer_type
    group by fleet_id
) as sub_d
on f.fleet_id = sub_d.fleet_id
where 
    f.total_vehicles >= 20 and
    f.product_code = 'NMC' and
    f.active_flag = 'Y' and
    m.mcs150_date < now() - interval '30 months'
""")
conn.commit()

## Deactivations based on missing DOT

In [30]:
# FH Fleets with 20+ vehicles and all PFDs that don't appear in MCMIS
# As of 1/30/2020 takes too long
# As of 1/31/2020 takes less than a minute! Change order of the joins to make MCMIS the last join

cur.execute("""
insert into fleetseek.update_output 
select
    '',
    m.icc1,
    m.act_stat,
    m.carship,
    m.census_num,
    m.dbnum,
    m.rating,
    m.chngdate,
    m.mcs150_date,
    m.class,
    m.classdef,
    m.name,
    m.name_dba,
    m.crrinter,
    m.crrhmintra,
    m.crrintra,
    m.shpinter,
    m.shpintra,
    m.org,
    m.genfreight,
    m.household,
    m.metalsheet,
    m.motorveh,
    m.drivetow,
    m.logpole,
    m.bldgmat,
    m.mobilehome,
    m.machlrg,
    m.produce,
    m.oilfield,
    m.liqgas,
    m.intermodal,
    m.passengers,
    m.livestock,
    m.grainfeed,
    m.coalcoke,
    m.meat,
    m.garbage,
    m.usmail,
    m.chem,
    m.drybulk,
    m.coldfood,
    m.beverages,
    m.paperprod,
    m.utility,
    m.farmsupp,
    m.construct,
    m.waterwell,
    m.cargoothr,
    m.othercargo,
    m.hm_ind,
    m.owntruck,
    m.owntract,
    m.owntrail,
    m.trmtruck,
    m.trmtract,
    m.trmtrail,
    m.trptruck,
    m.trptract,
    m.trptrail,
    m.tot_trucks,
    m.tot_buses,
    m.tot_pwr,
    m.phy_str,
    ubci.phy_address,
    m.phy_city,
    ubci.phy_city,
    m.phy_st,
    ubci.phy_state,
    m.phy_zip,
    ubci.phy_zip_code,
    m.phy_natn,
    m.phy_cnty,
    ubci.phy_countyname,
    ubci.phy_latitude,
    ubci.phy_longitude,
    m.undeliv_phy,
    m.mai_str,
    ubci.mai_address,
    m.mai_city,
    ubci.mai_city,
    m.mai_st,
    ubci.mai_state,
    m.mai_zip,
    ubci.mai_zip_code,
    m.mai_natn,
    m.mai_cnty,
    ubci.mai_countyname,
    ubci.mai_latitude,
    ubci.mai_longitude,
    m.undeliv_mai,
    m.tel_num,
    m.fax_num,
    m.email_address,
    f.average_miles,
    m.tot_drs,
    m.cdl_drs,
    m.mcs150mileageyear,
    m.mlg150,
    m.ratedate,
    m.adddate,
    m.acc_rate,
    m.company_rep1,
    m.company_rep2,
    f.fleet_id,
    f.usdot,
    f.mcn,
    f.sic_code,
    f.update_date,
    f.active_flag,
    sub_a.commodities,
    f.company_type_code,
    f.product_code,
    f.company_name,

    li.phy_address1,
    li.phy_address2,
    li.phy_address3,
    li.phy_state,
    li.phy_city,
    li.phy_country,
    li.phy_zip,
    li.phy_countyfips,
    li.phy_latitude,
    li.phy_longitude,
    li.phy_location_id,
    li.mai_address1,
    li.mai_address2,
    li.mai_address3,
    li.mai_state,
    li.mai_city,
    li.mai_country,
    li.mai_zip,
    li.mai_countyfips,
    li.mai_latitude,
    li.mai_longitude,
    li.mai_location_id,

    c.first_name,
    c.middle_name,
    c.last_name,
    c.suffix,
    c.original_job_title,
    c.contact_id,
    sub_b.first_name, 
    sub_b.middle_name,
    sub_b.last_name,
    sub_b.suffix,
    sub_b.original_job_title,
    sub_b.contact_id,
    sub_c.phone_number,
    sub_c.phone_number_id,
    sub_c.tf_number,
    sub_c.tf_number_id,
    sub_c.fax_number,
    sub_c.fax_number_id,
    e.email_address,
    e.email_id,
    f.trucks_owned,
    f.trucks_leased,
    f.haz_trucks_owned,
    f.haz_trucks_leased,
    f.tractors_owned,
    f.tractors_leased,
    f.total_vehicles,
    f.trailers_owned,
    f.trailers_leased,
    f.haz_trailers_owned,
    f.haz_trailers_leased,
    sub_d.trailer_type_id,
    f.drivers,
    f.cdl_drivers,
    f.gvw_class_12,
    f.gvw_class_345,
    f.gvw_class_6,
    f.gvw_class_7,
    f.gvw_class_8,
    f.revenue,
    f.website,
    'Inactive DOT',
    ''
from fleetseek.fleet as f
left join mcmis as m
on f.usdot = m.census_num
left join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
left join fleetseek.updates_bn_coder_input as ubci
on f.fleet_id = ubci.fs_fleet_id
left join (
    select 
        fp.fleet_id,
        l1.address1 as phy_address1,
        l1.address2 as phy_address2,
        l1.address3 as phy_address3,
        l1.state_code as phy_state,
        l1.city as phy_city,
        l1.country_code as phy_country,
        l1.zip_code as phy_zip,
        l1.county_fips as phy_countyfips,
        l1.latitude as phy_latitude,
        l1.longitude as phy_longitude,
        l1.location_id as phy_location_id,
        l2.address1 as mai_address1,
        l2.address2 as mai_address2,
        l2.address3 as mai_address3,
        l2.state_code as mai_state,
        l2.city as mai_city,
        l2.country_code as mai_country,
        l2.zip_code as mai_zip,
        l2.county_fips as mai_countyfips,
        l2.latitude as mai_latitude,
        l2.longitude as mai_longitude,
        l2.location_id as mai_location_id

    from fleetseek.location as l1
    join fleetseek.fleet_primary as fp
    on l1.location_id = fp.primary_physical_location_id
    join fleetseek.location as l2
    on fp.primary_mailing_location_id = l2.location_id

) as li
on f.fleet_id = li.fleet_id
left join fleetseek.contact as c
on fp.primary_contact_id = c.contact_id
left join(
    select 
        min(c.contact_id), 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id
    from fleetseek.fleet_contact as fc
    join fleetseek.fleet as f
    on fc.fleet_id = f.fleet_id
    join fleetseek.contact as c
    on fc.contact_id = c.contact_id
    where 
        sort_order = 2 and
        c.active = true 
    group by 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id

) as sub_b
on fp.fleet_id = sub_b.fleet_id
left join (
    select 
        fleet_id, 
        string_agg(commodity_id::varchar, ', ') as commodities
    from fleetseek.fleet_commodity
    group by fleet_id
) as sub_a
on f.fleet_id = sub_a.fleet_id
left join (
    select 
        pn.phone_number as phone_number, 
        pn.phone_number_id as phone_number_id, 
        pn2.phone_number as fax_number, 
        pn2.phone_number_id as fax_number_id,
        pn3.phone_number as tf_number, 
        pn3.phone_number_id as tf_number_id,
        fp.fleet_id
    from fleetseek.fleet_primary as fp
    left join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id 
    left join fleetseek.phone_number as pn2
    on fp.primary_fax_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_toll_free_id = pn3.phone_number_id
) as sub_c
on f.fleet_id = sub_c.fleet_id
left join fleetseek.email as e
on fp.primary_email_id = e.email_id
left join (
    select 
        fleet_id, 
        string_agg(trailer_type_id::varchar, ', ') as trailer_type_id
    from fleetseek.fleet_trailer_type
    group by fleet_id
) as sub_d
on f.fleet_id = sub_d.fleet_id
where 
    f.total_vehicles >= 20 and
    f.product_code = 'NMC' and
    f.active_flag = 'Y' and
    m.census_num is null or
    (f.product_code = 'PFD' and
    f.active_flag = 'Y' and
    m.census_num is null)
""")
conn.commit()

In [31]:
# FH Fleets with 20+ vehicles and all PFDs that don't appear in MCMIS
cur.execute("""

select
    '',
    m.icc1,
    m.act_stat,
    m.carship,
    m.census_num,
    m.dbnum,
    m.rating,
    m.chngdate,
    m.mcs150_date,
    m.class,
    m.classdef,
    m.name,
    m.name_dba,
    m.crrinter,
    m.crrhmintra,
    m.crrintra,
    m.shpinter,
    m.shpintra,
    m.org,
    m.genfreight,
    m.household,
    m.metalsheet,
    m.motorveh,
    m.drivetow,
    m.logpole,
    m.bldgmat,
    m.mobilehome,
    m.machlrg,
    m.produce,
    m.oilfield,
    m.liqgas,
    m.intermodal,
    m.passengers,
    m.livestock,
    m.grainfeed,
    m.coalcoke,
    m.meat,
    m.garbage,
    m.usmail,
    m.chem,
    m.drybulk,
    m.coldfood,
    m.beverages,
    m.paperprod,
    m.utility,
    m.farmsupp,
    m.construct,
    m.waterwell,
    m.cargoothr,
    m.othercargo,
    m.hm_ind,
    m.owntruck,
    m.owntract,
    m.owntrail,
    m.trmtruck,
    m.trmtract,
    m.trmtrail,
    m.trptruck,
    m.trptract,
    m.trptrail,
    m.tot_trucks,
    m.tot_buses,
    m.tot_pwr,
    m.phy_str,
    ubci.phy_address,
    m.phy_city,
    ubci.phy_city,
    m.phy_st,
    ubci.phy_state,
    m.phy_zip,
    ubci.phy_zip_code,
    m.phy_natn,
    m.phy_cnty,
    ubci.phy_countyname,
    ubci.phy_latitude,
    ubci.phy_longitude,
    m.undeliv_phy,
    m.mai_str,
    ubci.mai_address,
    m.mai_city,
    ubci.mai_city,
    m.mai_st,
    ubci.mai_state,
    m.mai_zip,
    ubci.mai_zip_code,
    m.mai_natn,
    m.mai_cnty,
    ubci.mai_countyname,
    ubci.mai_latitude,
    ubci.mai_longitude,
    m.undeliv_mai,
    m.tel_num,
    m.fax_num,
    m.email_address,
    f.average_miles,
    m.tot_drs,
    m.cdl_drs,
    m.mcs150mileageyear,
    m.mlg150,
    m.ratedate,
    m.adddate,
    m.acc_rate,
    m.company_rep1,
    m.company_rep2,
    f.fleet_id,
    f.usdot,
    f.mcn,
    f.sic_code,
    f.update_date,
    f.active_flag,
    sub_a.commodities,
    f.company_type_code,
    f.product_code,
    f.company_name,

    li.phy_address1,
    li.phy_address2,
    li.phy_address3,
    li.phy_state,
    li.phy_city,
    li.phy_country,
    li.phy_zip,
    li.phy_countyfips,
    li.phy_latitude,
    li.phy_longitude,
    li.phy_location_id,
    li.mai_address1,
    li.mai_address2,
    li.mai_address3,
    li.mai_state,
    li.mai_city,
    li.mai_country,
    li.mai_zip,
    li.mai_countyfips,
    li.mai_latitude,
    li.mai_longitude,
    li.mai_location_id,

    c.first_name,
    c.middle_name,
    c.last_name,
    c.suffix,
    c.original_job_title,
    c.contact_id,
    sub_b.first_name, 
    sub_b.middle_name,
    sub_b.last_name,
    sub_b.suffix,
    sub_b.original_job_title,
    sub_b.contact_id,
    sub_c.phone_number,
    sub_c.phone_number_id,
    sub_c.tf_number,
    sub_c.tf_number_id,
    sub_c.fax_number,
    sub_c.fax_number_id,
    e.email_address,
    e.email_id,
    f.trucks_owned,
    f.trucks_leased,
    f.haz_trucks_owned,
    f.haz_trucks_leased,
    f.tractors_owned,
    f.tractors_leased,
    f.total_vehicles,
    f.trailers_owned,
    f.trailers_leased,
    f.haz_trailers_owned,
    f.haz_trailers_leased,
    sub_d.trailer_type_id,
    f.drivers,
    f.cdl_drivers,
    f.gvw_class_12,
    f.gvw_class_345,
    f.gvw_class_6,
    f.gvw_class_7,
    f.gvw_class_8,
    f.revenue,
    f.website,
    'Inactive DOT',
    ''
from fleetseek.fleet as f
left join mcmis as m
on f.usdot = m.census_num
left join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
left join fleetseek.updates_bn_coder_input as ubci
on f.fleet_id = ubci.fs_fleet_id
left join (
    select 
        fp.fleet_id,
        l1.address1 as phy_address1,
        l1.address2 as phy_address2,
        l1.address3 as phy_address3,
        l1.state_code as phy_state,
        l1.city as phy_city,
        l1.country_code as phy_country,
        l1.zip_code as phy_zip,
        l1.county_fips as phy_countyfips,
        l1.latitude as phy_latitude,
        l1.longitude as phy_longitude,
        l1.location_id as phy_location_id,
        l2.address1 as mai_address1,
        l2.address2 as mai_address2,
        l2.address3 as mai_address3,
        l2.state_code as mai_state,
        l2.city as mai_city,
        l2.country_code as mai_country,
        l2.zip_code as mai_zip,
        l2.county_fips as mai_countyfips,
        l2.latitude as mai_latitude,
        l2.longitude as mai_longitude,
        l2.location_id as mai_location_id

    from fleetseek.location as l1
    join fleetseek.fleet_primary as fp
    on l1.location_id = fp.primary_physical_location_id
    join fleetseek.location as l2
    on fp.primary_mailing_location_id = l2.location_id

) as li
on f.fleet_id = li.fleet_id
left join fleetseek.contact as c
on fp.primary_contact_id = c.contact_id
left join(
    select 
        min(c.contact_id), 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id
    from fleetseek.fleet_contact as fc
    join fleetseek.fleet as f
    on fc.fleet_id = f.fleet_id
    join fleetseek.contact as c
    on fc.contact_id = c.contact_id
    where 
        sort_order = 2 and
        c.active = true 
    group by 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id

) as sub_b
on fp.fleet_id = sub_b.fleet_id
left join (
    select 
        fleet_id, 
        string_agg(commodity_id::varchar, ', ') as commodities
    from fleetseek.fleet_commodity
    group by fleet_id
) as sub_a
on f.fleet_id = sub_a.fleet_id
left join (
    select 
        pn.phone_number as phone_number, 
        pn.phone_number_id as phone_number_id, 
        pn2.phone_number as fax_number, 
        pn2.phone_number_id as fax_number_id,
        pn3.phone_number as tf_number, 
        pn3.phone_number_id as tf_number_id,
        fp.fleet_id
    from fleetseek.fleet_primary as fp
    left join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id 
    left join fleetseek.phone_number as pn2
    on fp.primary_fax_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_toll_free_id = pn3.phone_number_id
) as sub_c
on f.fleet_id = sub_c.fleet_id
left join fleetseek.email as e
on fp.primary_email_id = e.email_id
left join (
    select 
        fleet_id, 
        string_agg(trailer_type_id::varchar, ', ') as trailer_type_id
    from fleetseek.fleet_trailer_type
    group by fleet_id
) as sub_d
on f.fleet_id = sub_d.fleet_id
where 
    f.total_vehicles >= 20 and
    f.product_code = 'NMC' and
    f.active_flag = 'Y' and
    m.census_num is null or
    (f.product_code = 'PFD' and
    f.active_flag = 'Y' and
    m.census_num is null)
""")
conn.commit()

## Reinstatement for FH

In [32]:
# FH fleets with 50+ vehicles in the FSK database will always go to an analyst to confirm reinstatement 

cur.execute("""
insert into fleetseek.update_output 
select
    '',
    m.icc1,
    m.act_stat,
    m.carship,
    m.census_num,
    m.dbnum,
    m.rating,
    m.chngdate,
    m.mcs150_date,
    m.class,
    m.classdef,
    m.name,
    m.name_dba,
    m.crrinter,
    m.crrhmintra,
    m.crrintra,
    m.shpinter,
    m.shpintra,
    m.org,
    m.genfreight,
    m.household,
    m.metalsheet,
    m.motorveh,
    m.drivetow,
    m.logpole,
    m.bldgmat,
    m.mobilehome,
    m.machlrg,
    m.produce,
    m.oilfield,
    m.liqgas,
    m.intermodal,
    m.passengers,
    m.livestock,
    m.grainfeed,
    m.coalcoke,
    m.meat,
    m.garbage,
    m.usmail,
    m.chem,
    m.drybulk,
    m.coldfood,
    m.beverages,
    m.paperprod,
    m.utility,
    m.farmsupp,
    m.construct,
    m.waterwell,
    m.cargoothr,
    m.othercargo,
    m.hm_ind,
    m.owntruck,
    m.owntract,
    m.owntrail,
    m.trmtruck,
    m.trmtract,
    m.trmtrail,
    m.trptruck,
    m.trptract,
    m.trptrail,
    m.tot_trucks,
    m.tot_buses,
    m.tot_pwr,
    m.phy_str,
    ubci.phy_address,
    m.phy_city,
    ubci.phy_city,
    m.phy_st,
    ubci.phy_state,
    m.phy_zip,
    ubci.phy_zip_code,
    m.phy_natn,
    m.phy_cnty,
    ubci.phy_countyname,
    ubci.phy_latitude,
    ubci.phy_longitude,
    m.undeliv_phy,
    m.mai_str,
    ubci.mai_address,
    m.mai_city,
    ubci.mai_city,
    m.mai_st,
    ubci.mai_state,
    m.mai_zip,
    ubci.mai_zip_code,
    m.mai_natn,
    m.mai_cnty,
    ubci.mai_countyname,
    ubci.mai_latitude,
    ubci.mai_longitude,
    m.undeliv_mai,
    m.tel_num,
    m.fax_num,
    m.email_address,
    f.average_miles,
    m.tot_drs,
    m.cdl_drs,
    m.mcs150mileageyear,
    m.mlg150,
    m.ratedate,
    m.adddate,
    m.acc_rate,
    m.company_rep1,
    m.company_rep2,
    f.fleet_id,
    f.usdot,
    f.mcn,
    f.sic_code,
    f.update_date,
    f.active_flag,
    sub_a.commodities,
    f.company_type_code,
    f.product_code,
    f.company_name,

    li.phy_address1,
    li.phy_address2,
    li.phy_address3,
    li.phy_state,
    li.phy_city,
    li.phy_country,
    li.phy_zip,
    li.phy_countyfips,
    li.phy_latitude,
    li.phy_longitude,
    li.phy_location_id,
    li.mai_address1,
    li.mai_address2,
    li.mai_address3,
    li.mai_state,
    li.mai_city,
    li.mai_country,
    li.mai_zip,
    li.mai_countyfips,
    li.mai_latitude,
    li.mai_longitude,
    li.mai_location_id,

    c.first_name,
    c.middle_name,
    c.last_name,
    c.suffix,
    c.original_job_title,
    c.contact_id,
    sub_b.first_name, 
    sub_b.middle_name,
    sub_b.last_name,
    sub_b.suffix,
    sub_b.original_job_title,
    sub_b.contact_id,
    sub_c.phone_number,
    sub_c.phone_number_id,
    sub_c.tf_number,
    sub_c.tf_number_id,
    sub_c.fax_number,
    sub_c.fax_number_id,
    e.email_address,
    e.email_id,
    f.trucks_owned,
    f.trucks_leased,
    f.haz_trucks_owned,
    f.haz_trucks_leased,
    f.tractors_owned,
    f.tractors_leased,
    f.total_vehicles,
    f.trailers_owned,
    f.trailers_leased,
    f.haz_trailers_owned,
    f.haz_trailers_leased,
    sub_d.trailer_type_id,
    f.drivers,
    f.cdl_drivers,
    f.gvw_class_12,
    f.gvw_class_345,
    f.gvw_class_6,
    f.gvw_class_7,
    f.gvw_class_8,
    f.revenue,
    f.website,
    'For-Hire Reinstatement',
    ''
from fleetseek.fleet as f
join mcmis as m
on f.usdot = m.census_num
left join fleetseek.updates_bn_coder_input as ubci
on f.fleet_id = ubci.fs_fleet_id
left join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
left join (
    select 
        fp.fleet_id,
        l1.address1 as phy_address1,
        l1.address2 as phy_address2,
        l1.address3 as phy_address3,
        l1.state_code as phy_state,
        l1.city as phy_city,
        l1.country_code as phy_country,
        l1.zip_code as phy_zip,
        l1.county_fips as phy_countyfips,
        l1.latitude as phy_latitude,
        l1.longitude as phy_longitude,
        l1.location_id as phy_location_id,
        l2.address1 as mai_address1,
        l2.address2 as mai_address2,
        l2.address3 as mai_address3,
        l2.state_code as mai_state,
        l2.city as mai_city,
        l2.country_code as mai_country,
        l2.zip_code as mai_zip,
        l2.county_fips as mai_countyfips,
        l2.latitude as mai_latitude,
        l2.longitude as mai_longitude,
        l2.location_id as mai_location_id

    from fleetseek.location as l1
    join fleetseek.fleet_primary as fp
    on l1.location_id = fp.primary_physical_location_id
    join fleetseek.location as l2
    on fp.primary_mailing_location_id = l2.location_id

) as li
on f.fleet_id = li.fleet_id
left join fleetseek.contact as c
on fp.primary_contact_id = c.contact_id
left join(
    select 
        min(c.contact_id), 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id
    from fleetseek.fleet_contact as fc
    join fleetseek.fleet as f
    on fc.fleet_id = f.fleet_id
    join fleetseek.contact as c
    on fc.contact_id = c.contact_id
    where 
        sort_order = 2 and
        c.active = true 
    group by 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id

) as sub_b
on fp.fleet_id = sub_b.fleet_id
left join (
    select 
        fleet_id, 
        string_agg(commodity_id::varchar, ', ') as commodities
    from fleetseek.fleet_commodity
    group by fleet_id
) as sub_a
on f.fleet_id = sub_a.fleet_id
left join (
    select 
        pn.phone_number as phone_number, 
        pn.phone_number_id as phone_number_id, 
        pn2.phone_number as fax_number, 
        pn2.phone_number_id as fax_number_id,
        pn3.phone_number as tf_number, 
        pn3.phone_number_id as tf_number_id,
        fp.fleet_id
    from fleetseek.fleet_primary as fp
    left join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id 
    left join fleetseek.phone_number as pn2
    on fp.primary_fax_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_toll_free_id = pn3.phone_number_id
) as sub_c
on f.fleet_id = sub_c.fleet_id
left join fleetseek.email as e
on fp.primary_email_id = e.email_id
left join (
    select 
        fleet_id, 
        string_agg(trailer_type_id::varchar, ', ') as trailer_type_id
    from fleetseek.fleet_trailer_type
    group by fleet_id
) as sub_d
on f.fleet_id = sub_d.fleet_id
where 
    f.total_vehicles >= 50 and
    f.product_code = 'NMC' and
    f.active_flag = 'N' and
    m.mcs150_date > f.update_date and
    m.mcs150_date > now() - interval '2 years'

""")
conn.commit()

In [33]:
# FH and OO fleets that have less than 50 total vehicles but show growth of more than 50 total vehicles will go to an analyst

cur.execute("""
insert into fleetseek.update_output 
select
    '',
    m.icc1,
    m.act_stat,
    m.carship,
    m.census_num,
    m.dbnum,
    m.rating,
    m.chngdate,
    m.mcs150_date,
    m.class,
    m.classdef,
    m.name,
    m.name_dba,
    m.crrinter,
    m.crrhmintra,
    m.crrintra,
    m.shpinter,
    m.shpintra,
    m.org,
    m.genfreight,
    m.household,
    m.metalsheet,
    m.motorveh,
    m.drivetow,
    m.logpole,
    m.bldgmat,
    m.mobilehome,
    m.machlrg,
    m.produce,
    m.oilfield,
    m.liqgas,
    m.intermodal,
    m.passengers,
    m.livestock,
    m.grainfeed,
    m.coalcoke,
    m.meat,
    m.garbage,
    m.usmail,
    m.chem,
    m.drybulk,
    m.coldfood,
    m.beverages,
    m.paperprod,
    m.utility,
    m.farmsupp,
    m.construct,
    m.waterwell,
    m.cargoothr,
    m.othercargo,
    m.hm_ind,
    m.owntruck,
    m.owntract,
    m.owntrail,
    m.trmtruck,
    m.trmtract,
    m.trmtrail,
    m.trptruck,
    m.trptract,
    m.trptrail,
    m.tot_trucks,
    m.tot_buses,
    m.tot_pwr,
    m.phy_str,
    ubci.phy_address,
    m.phy_city,
    ubci.phy_city,
    m.phy_st,
    ubci.phy_state,
    m.phy_zip,
    ubci.phy_zip_code,
    m.phy_natn,
    m.phy_cnty,
    ubci.phy_countyname,
    ubci.phy_latitude,
    ubci.phy_longitude,
    m.undeliv_phy,
    m.mai_str,
    ubci.mai_address,
    m.mai_city,
    ubci.mai_city,
    m.mai_st,
    ubci.mai_state,
    m.mai_zip,
    ubci.mai_zip_code,
    m.mai_natn,
    m.mai_cnty,
    ubci.mai_countyname,
    ubci.mai_latitude,
    ubci.mai_longitude,
    m.undeliv_mai,
    m.tel_num,
    m.fax_num,
    m.email_address,
    f.average_miles,
    m.tot_drs,
    m.cdl_drs,
    m.mcs150mileageyear,
    m.mlg150,
    m.ratedate,
    m.adddate,
    m.acc_rate,
    m.company_rep1,
    m.company_rep2,
    f.fleet_id,
    f.usdot,
    f.mcn,
    f.sic_code,
    f.update_date,
    f.active_flag,
    sub_a.commodities,
    f.company_type_code,
    f.product_code,
    f.company_name,

    li.phy_address1,
    li.phy_address2,
    li.phy_address3,
    li.phy_state,
    li.phy_city,
    li.phy_country,
    li.phy_zip,
    li.phy_countyfips,
    li.phy_latitude,
    li.phy_longitude,
    li.phy_location_id,
    li.mai_address1,
    li.mai_address2,
    li.mai_address3,
    li.mai_state,
    li.mai_city,
    li.mai_country,
    li.mai_zip,
    li.mai_countyfips,
    li.mai_latitude,
    li.mai_longitude,
    li.mai_location_id,

    c.first_name,
    c.middle_name,
    c.last_name,
    c.suffix,
    c.original_job_title,
    c.contact_id,
    sub_b.first_name, 
    sub_b.middle_name,
    sub_b.last_name,
    sub_b.suffix,
    sub_b.original_job_title,
    sub_b.contact_id,
    sub_c.phone_number,
    sub_c.phone_number_id,
    sub_c.tf_number,
    sub_c.tf_number_id,
    sub_c.fax_number,
    sub_c.fax_number_id,
    e.email_address,
    e.email_id,
    f.trucks_owned,
    f.trucks_leased,
    f.haz_trucks_owned,
    f.haz_trucks_leased,
    f.tractors_owned,
    f.tractors_leased,
    f.total_vehicles,
    f.trailers_owned,
    f.trailers_leased,
    f.haz_trailers_owned,
    f.haz_trailers_leased,
    sub_d.trailer_type_id,
    f.drivers,
    f.cdl_drivers,
    f.gvw_class_12,
    f.gvw_class_345,
    f.gvw_class_6,
    f.gvw_class_7,
    f.gvw_class_8,
    f.revenue,
    f.website,
    'For-Hire Reinstatement, (growth > 50)',
    ''
from fleetseek.fleet as f
join mcmis as m
on f.usdot = m.census_num
join (
    select 
        f.fleet_id, 
        f.total_vehicles,
        f.product_code,
        f.active_flag,
        m.mcs150_date,
        f.update_date,

        sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract) as mcmis_total_vehicles
    from fleetseek.fleet as f
    join mcmis as m
    on f.usdot = m.census_num
    where

        f.active_flag = 'N' and
        m.mcs150_date > f.update_date and
        m.mcs150_date > now() - interval '2 years' and
        f.total_vehicles < 50 and
        f.product_code in ('OOD', 'NMC')
    group by 
        f.fleet_id, 
        f.total_vehicles,
        f.product_code,
        f.active_flag,
        m.mcs150_date,
        f.update_date,
        m.owntruck,
        m.trmtruck,
        m.trptruck,
        m.owntract,
        m.trmtract,
        m.trptract
    having abs(f.total_vehicles - sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract)) >= 50
) as sub_e
on f.fleet_id = sub_e.fleet_id
left join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
left join (
    select 
        fp.fleet_id,
        l1.address1 as phy_address1,
        l1.address2 as phy_address2,
        l1.address3 as phy_address3,
        l1.state_code as phy_state,
        l1.city as phy_city,
        l1.country_code as phy_country,
        l1.zip_code as phy_zip,
        l1.county_fips as phy_countyfips,
        l1.latitude as phy_latitude,
        l1.longitude as phy_longitude,
        l1.location_id as phy_location_id,
        l2.address1 as mai_address1,
        l2.address2 as mai_address2,
        l2.address3 as mai_address3,
        l2.state_code as mai_state,
        l2.city as mai_city,
        l2.country_code as mai_country,
        l2.zip_code as mai_zip,
        l2.county_fips as mai_countyfips,
        l2.latitude as mai_latitude,
        l2.longitude as mai_longitude,
        l2.location_id as mai_location_id

    from fleetseek.location as l1
    join fleetseek.fleet_primary as fp
    on l1.location_id = fp.primary_physical_location_id
    join fleetseek.location as l2
    on fp.primary_mailing_location_id = l2.location_id

) as li
on f.fleet_id = li.fleet_id
left join fleetseek.contact as c
on fp.primary_contact_id = c.contact_id
left join (
    select 
        fleet_id, 
        string_agg(commodity_id::varchar, ', ') as commodities
    from fleetseek.fleet_commodity
    group by fleet_id
) as sub_a
on f.fleet_id = sub_a.fleet_id
left join (
    select 
        pn.phone_number as phone_number, 
        pn.phone_number_id as phone_number_id, 
        pn2.phone_number as fax_number, 
        pn2.phone_number_id as fax_number_id,
        pn3.phone_number as tf_number, 
        pn3.phone_number_id as tf_number_id,
        fp.fleet_id
    from fleetseek.fleet_primary as fp
    left join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id 
    left join fleetseek.phone_number as pn2
    on fp.primary_fax_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_toll_free_id = pn3.phone_number_id
) as sub_c
on f.fleet_id = sub_c.fleet_id
left join fleetseek.updates_bn_coder_input as ubci
on f.fleet_id = ubci.fs_fleet_id
left join(
    select 
        min(c.contact_id), 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id
    from fleetseek.fleet_contact as fc
    join fleetseek.fleet as f
    on fc.fleet_id = f.fleet_id
    join fleetseek.contact as c
    on fc.contact_id = c.contact_id
    where 
        sort_order = 2 and
        c.active = true 
    group by 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id	
) as sub_b
on fp.fleet_id = sub_b.fleet_id
left join fleetseek.email as e
on fp.primary_email_id = e.email_id
left join (
    select 
        fleet_id, 
        string_agg(trailer_type_id::varchar, ', ') as trailer_type_id
    from fleetseek.fleet_trailer_type
    group by fleet_id
) as sub_d
on f.fleet_id = sub_d.fleet_id


""")
conn.commit()

## Reinstatement for PFD

In [34]:
# As of 1/29/2020 all PFDs will go to an analyst for reinstatement regardless of size of fleet
cur.execute(""" 
insert into fleetseek.update_output 
select
    '',
    m.icc1,
    m.act_stat,
    m.carship,
    m.census_num,
    m.dbnum,
    m.rating,
    m.chngdate,
    m.mcs150_date,
    m.class,
    m.classdef,
    m.name,
    m.name_dba,
    m.crrinter,
    m.crrhmintra,
    m.crrintra,
    m.shpinter,
    m.shpintra,
    m.org,
    m.genfreight,
    m.household,
    m.metalsheet,
    m.motorveh,
    m.drivetow,
    m.logpole,
    m.bldgmat,
    m.mobilehome,
    m.machlrg,
    m.produce,
    m.oilfield,
    m.liqgas,
    m.intermodal,
    m.passengers,
    m.livestock,
    m.grainfeed,
    m.coalcoke,
    m.meat,
    m.garbage,
    m.usmail,
    m.chem,
    m.drybulk,
    m.coldfood,
    m.beverages,
    m.paperprod,
    m.utility,
    m.farmsupp,
    m.construct,
    m.waterwell,
    m.cargoothr,
    m.othercargo,
    m.hm_ind,
    m.owntruck,
    m.owntract,
    m.owntrail,
    m.trmtruck,
    m.trmtract,
    m.trmtrail,
    m.trptruck,
    m.trptract,
    m.trptrail,
    m.tot_trucks,
    m.tot_buses,
    m.tot_pwr,
    m.phy_str,
    ubci.phy_address,
    m.phy_city,
    ubci.phy_city,
    m.phy_st,
    ubci.phy_state,
    m.phy_zip,
    ubci.phy_zip_code,
    m.phy_natn,
    m.phy_cnty,
    ubci.phy_countyname,
    ubci.phy_latitude,
    ubci.phy_longitude,
    m.undeliv_phy,
    m.mai_str,
    ubci.mai_address,
    m.mai_city,
    ubci.mai_city,
    m.mai_st,
    ubci.mai_state,
    m.mai_zip,
    ubci.mai_zip_code,
    m.mai_natn,
    m.mai_cnty,
    ubci.mai_countyname,
    ubci.mai_latitude,
    ubci.mai_longitude,
    m.undeliv_mai,
    m.tel_num,
    m.fax_num,
    m.email_address,
    f.average_miles,
    m.tot_drs,
    m.cdl_drs,
    m.mcs150mileageyear,
    m.mlg150,
    m.ratedate,
    m.adddate,
    m.acc_rate,
    m.company_rep1,
    m.company_rep2,
    f.fleet_id,
    f.usdot,
    f.mcn,
    f.sic_code,
    f.update_date,
    f.active_flag,
    sub_a.commodities,
    f.company_type_code,
    f.product_code,
    f.company_name,

    li.phy_address1,
    li.phy_address2,
    li.phy_address3,
    li.phy_state,
    li.phy_city,
    li.phy_country,
    li.phy_zip,
    li.phy_countyfips,
    li.phy_latitude,
    li.phy_longitude,
    li.phy_location_id,
    li.mai_address1,
    li.mai_address2,
    li.mai_address3,
    li.mai_state,
    li.mai_city,
    li.mai_country,
    li.mai_zip,
    li.mai_countyfips,
    li.mai_latitude,
    li.mai_longitude,
    li.mai_location_id,

    c.first_name,
    c.middle_name,
    c.last_name,
    c.suffix,
    c.original_job_title,
    c.contact_id,
    sub_b.first_name, 
    sub_b.middle_name,
    sub_b.last_name,
    sub_b.suffix,
    sub_b.original_job_title,
    sub_b.contact_id,
    sub_c.phone_number,
    sub_c.phone_number_id,
    sub_c.tf_number,
    sub_c.tf_number_id,
    sub_c.fax_number,
    sub_c.fax_number_id,
    e.email_address,
    e.email_id,
    f.trucks_owned,
    f.trucks_leased,
    f.haz_trucks_owned,
    f.haz_trucks_leased,
    f.tractors_owned,
    f.tractors_leased,
    f.total_vehicles,
    f.trailers_owned,
    f.trailers_leased,
    f.haz_trailers_owned,
    f.haz_trailers_leased,
    sub_d.trailer_type_id,
    f.drivers,
    f.cdl_drivers,
    f.gvw_class_12,
    f.gvw_class_345,
    f.gvw_class_6,
    f.gvw_class_7,
    f.gvw_class_8,
    f.revenue,
    f.website,
    'Private Fleet Reinstatement',
    ''
from fleetseek.fleet as f
join mcmis as m
on f.usdot = m.census_num
left join fleetseek.updates_bn_coder_input as ubci
on f.fleet_id = ubci.fs_fleet_id
left join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
left join (
    select 
        fp.fleet_id,
        l1.address1 as phy_address1,
        l1.address2 as phy_address2,
        l1.address3 as phy_address3,
        l1.state_code as phy_state,
        l1.city as phy_city,
        l1.country_code as phy_country,
        l1.zip_code as phy_zip,
        l1.county_fips as phy_countyfips,
        l1.latitude as phy_latitude,
        l1.longitude as phy_longitude,
        l1.location_id as phy_location_id,
        l2.address1 as mai_address1,
        l2.address2 as mai_address2,
        l2.address3 as mai_address3,
        l2.state_code as mai_state,
        l2.city as mai_city,
        l2.country_code as mai_country,
        l2.zip_code as mai_zip,
        l2.county_fips as mai_countyfips,
        l2.latitude as mai_latitude,
        l2.longitude as mai_longitude,
        l2.location_id as mai_location_id

    from fleetseek.location as l1
    join fleetseek.fleet_primary as fp
    on l1.location_id = fp.primary_physical_location_id
    join fleetseek.location as l2
    on fp.primary_mailing_location_id = l2.location_id

) as li
on f.fleet_id = li.fleet_id
left join fleetseek.contact as c
on fp.primary_contact_id = c.contact_id
left join(
    select 
        min(c.contact_id), 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id
    from fleetseek.fleet_contact as fc
    join fleetseek.fleet as f
    on fc.fleet_id = f.fleet_id
    join fleetseek.contact as c
    on fc.contact_id = c.contact_id
    where 
        sort_order = 2 and
        c.active = true 
    group by 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id

) as sub_b
on fp.fleet_id = sub_b.fleet_id
left join (
    select 
        fleet_id, 
        string_agg(commodity_id::varchar, ', ') as commodities
    from fleetseek.fleet_commodity
    group by fleet_id
) as sub_a
on f.fleet_id = sub_a.fleet_id
left join (
    select 
        pn.phone_number as phone_number, 
        pn.phone_number_id as phone_number_id, 
        pn2.phone_number as fax_number, 
        pn2.phone_number_id as fax_number_id,
        pn3.phone_number as tf_number, 
        pn3.phone_number_id as tf_number_id,
        fp.fleet_id
    from fleetseek.fleet_primary as fp
    join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id 
    left join fleetseek.phone_number as pn2
    on fp.primary_fax_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_toll_free_id = pn3.phone_number_id
) as sub_c
on f.fleet_id = sub_c.fleet_id
left join fleetseek.email as e
on fp.primary_email_id = e.email_id
left join (
    select 
        fleet_id, 
        string_agg(trailer_type_id::varchar, ', ') as trailer_type_id
    from fleetseek.fleet_trailer_type
    group by fleet_id
) as sub_d
on f.fleet_id = sub_d.fleet_id
where 
    f.product_code = 'PFD' and
    f.active_flag = 'N' and
    m.mcs150_date > f.update_date and
    m.mcs150_date > now() - interval '2 years'

""")
conn.commit()

## Company name changes

In [35]:
# Company name differentiations for companies that are greater than 20+ total_vehicles
cur.execute("""
insert into fleetseek.update_output 

select
    '',
    m.icc1,
    m.act_stat,
    m.carship,
    m.census_num,
    m.dbnum,
    m.rating,
    m.chngdate,
    m.mcs150_date,
    m.class,
    m.classdef,
    m.name,
    m.name_dba,
    m.crrinter,
    m.crrhmintra,
    m.crrintra,
    m.shpinter,
    m.shpintra,
    m.org,
    m.genfreight,
    m.household,
    m.metalsheet,
    m.motorveh,
    m.drivetow,
    m.logpole,
    m.bldgmat,
    m.mobilehome,
    m.machlrg,
    m.produce,
    m.oilfield,
    m.liqgas,
    m.intermodal,
    m.passengers,
    m.livestock,
    m.grainfeed,
    m.coalcoke,
    m.meat,
    m.garbage,
    m.usmail,
    m.chem,
    m.drybulk,
    m.coldfood,
    m.beverages,
    m.paperprod,
    m.utility,
    m.farmsupp,
    m.construct,
    m.waterwell,
    m.cargoothr,
    m.othercargo,
    m.hm_ind,
    m.owntruck,
    m.owntract,
    m.owntrail,
    m.trmtruck,
    m.trmtract,
    m.trmtrail,
    m.trptruck,
    m.trptract,
    m.trptrail,
    m.tot_trucks,
    m.tot_buses,
    m.tot_pwr,
    m.phy_str,
    ubci.phy_address,
    m.phy_city,
    ubci.phy_city,
    m.phy_st,
    ubci.phy_state,
    m.phy_zip,
    ubci.phy_zip_code,
    m.phy_natn,
    m.phy_cnty,
    ubci.phy_countyname,
    ubci.phy_latitude,
    ubci.phy_longitude,
    m.undeliv_phy,
    m.mai_str,
    ubci.mai_address,
    m.mai_city,
    ubci.mai_city,
    m.mai_st,
    ubci.mai_state,
    m.mai_zip,
    ubci.mai_zip_code,
    m.mai_natn,
    m.mai_cnty,
    ubci.mai_countyname,
    ubci.mai_latitude,
    ubci.mai_longitude,
    m.undeliv_mai,
    m.tel_num,
    m.fax_num,
    m.email_address,
    f.average_miles,
    m.tot_drs,
    m.cdl_drs,
    m.mcs150mileageyear,
    m.mlg150,
    m.ratedate,
    m.adddate,
    m.acc_rate,
    m.company_rep1,
    m.company_rep2,
    f.fleet_id,
    f.usdot,
    f.mcn,
    f.sic_code,
    f.update_date,
    f.active_flag,
    sub_a.commodities,
    f.company_type_code,
    f.product_code,
    f.company_name,

    li.phy_address1,
    li.phy_address2,
    li.phy_address3,
    li.phy_state,
    li.phy_city,
    li.phy_country,
    li.phy_zip,
    li.phy_countyfips,
    li.phy_latitude,
    li.phy_longitude,
    li.phy_location_id,
    li.mai_address1,
    li.mai_address2,
    li.mai_address3,
    li.mai_state,
    li.mai_city,
    li.mai_country,
    li.mai_zip,
    li.mai_countyfips,
    li.mai_latitude,
    li.mai_longitude,
    li.mai_location_id,

    c.first_name,
    c.middle_name,
    c.last_name,
    c.suffix,
    c.original_job_title,
    c.contact_id,
    sub_b.first_name, 
    sub_b.middle_name,
    sub_b.last_name,
    sub_b.suffix,
    sub_b.original_job_title,
    sub_b.contact_id,
    sub_c.phone_number,
    sub_c.phone_number_id,
    sub_c.tf_number,
    sub_c.tf_number_id,
    sub_c.fax_number,
    sub_c.fax_number_id,
    e.email_address,
    e.email_id,
    f.trucks_owned,
    f.trucks_leased,
    f.haz_trucks_owned,
    f.haz_trucks_leased,
    f.tractors_owned,
    f.tractors_leased,
    f.total_vehicles,
    f.trailers_owned,
    f.trailers_leased,
    f.haz_trailers_owned,
    f.haz_trailers_leased,
    sub_d.trailer_type_id,
    f.drivers,
    f.cdl_drivers,
    f.gvw_class_12,
    f.gvw_class_345,
    f.gvw_class_6,
    f.gvw_class_7,
    f.gvw_class_8,
    f.revenue,
    f.website,
    'Company Name Update',
    ''

from fleetseek.fleet as f
join mcmis as m
on f.usdot = m.census_num

left join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
left join (
    select 
        fp.fleet_id,
        l1.address1 as phy_address1,
        l1.address2 as phy_address2,
        l1.address3 as phy_address3,
        l1.state_code as phy_state,
        l1.city as phy_city,
        l1.country_code as phy_country,
        l1.zip_code as phy_zip,
        l1.county_fips as phy_countyfips,
        l1.latitude as phy_latitude,
        l1.longitude as phy_longitude,
        l1.location_id as phy_location_id,
        l2.address1 as mai_address1,
        l2.address2 as mai_address2,
        l2.address3 as mai_address3,
        l2.state_code as mai_state,
        l2.city as mai_city,
        l2.country_code as mai_country,
        l2.zip_code as mai_zip,
        l2.county_fips as mai_countyfips,
        l2.latitude as mai_latitude,
        l2.longitude as mai_longitude,
        l2.location_id as mai_location_id

    from fleetseek.location as l1
    join fleetseek.fleet_primary as fp
    on l1.location_id = fp.primary_physical_location_id
    join fleetseek.location as l2
    on fp.primary_mailing_location_id = l2.location_id

) as li
on f.fleet_id = li.fleet_id
left join fleetseek.updates_bn_coder_input as ubci
on f.fleet_id = ubci.fs_fleet_id
left join fleetseek.contact as c
on fp.primary_contact_id = c.contact_id
left join(
    select min(c.contact_id), fc.fleet_id, sort_order, c.first_name, c.middle_name, c.last_name, c.suffix, c.original_job_title, c.contact_id
    from fleetseek.fleet_contact as fc
    join fleetseek.fleet as f
    on fc.fleet_id = f.fleet_id
    join fleetseek.contact as c
    on fc.contact_id = c.contact_id
    where sort_order = 2 and
    c.active = true 
    group by fc.fleet_id, sort_order, c.first_name, c.middle_name, c.last_name, c.suffix, c.original_job_title, c.contact_id

) as sub_b
on fp.fleet_id = sub_b.fleet_id
left join (
    select fleet_id, string_agg(commodity_id::varchar, ', ') as commodities
    from fleetseek.fleet_commodity
    group by fleet_id
) as sub_a
on f.fleet_id = sub_a.fleet_id
left join (
    select 
        pn.phone_number as phone_number, 
        pn.phone_number_id as phone_number_id, 
        pn2.phone_number as fax_number, 
        pn2.phone_number_id as fax_number_id,
        pn3.phone_number as tf_number, 
        pn3.phone_number_id as tf_number_id,
        fp.fleet_id
    from fleetseek.fleet_primary as fp
    left join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id 
    left join fleetseek.phone_number as pn2
    on fp.primary_fax_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_toll_free_id = pn3.phone_number_id
) as sub_c
on f.fleet_id = sub_c.fleet_id
left join fleetseek.email as e
on fp.primary_email_id = e.email_id
left join (
    select 
        fleet_id, 
        string_agg(trailer_type_id::varchar, ', ') as trailer_type_id
    from fleetseek.fleet_trailer_type
    group by fleet_id
) as sub_d
on f.fleet_id = sub_d.fleet_id
where 
    f.product_code in ('NMC', 'PFD') and
    f.active_flag = 'Y' and
    m.mcs150_date > f.update_date and
    m.mcs150_date > now() - interval '2 years' and
    f.total_vehicles >= 20 and
    (left(upper(f.company_name), 4) != left(m.name, 4) and left(upper(f.company_name), 4) != left(m.name_dba, 4))

""")
conn.commit()

## Addresses

In [36]:
# Greater than 20, if there is a different state in address. 
# 50 or greater if there is a different address in general
# If a fleet between 20-49 has the same state but a different address it will automatically be updated
cur.execute("""
insert into fleetseek.update_output 
select
    '',
    m.icc1,
    m.act_stat,
    m.carship,
    m.census_num,
    m.dbnum,
    m.rating,
    m.chngdate,
    m.mcs150_date,
    m.class,
    m.classdef,
    m.name,
    m.name_dba,
    m.crrinter,
    m.crrhmintra,
    m.crrintra,
    m.shpinter,
    m.shpintra,
    m.org,
    m.genfreight,
    m.household,
    m.metalsheet,
    m.motorveh,
    m.drivetow,
    m.logpole,
    m.bldgmat,
    m.mobilehome,
    m.machlrg,
    m.produce,
    m.oilfield,
    m.liqgas,
    m.intermodal,
    m.passengers,
    m.livestock,
    m.grainfeed,
    m.coalcoke,
    m.meat,
    m.garbage,
    m.usmail,
    m.chem,
    m.drybulk,
    m.coldfood,
    m.beverages,
    m.paperprod,
    m.utility,
    m.farmsupp,
    m.construct,
    m.waterwell,
    m.cargoothr,
    m.othercargo,
    m.hm_ind,
    m.owntruck,
    m.owntract,
    m.owntrail,
    m.trmtruck,
    m.trmtract,
    m.trmtrail,
    m.trptruck,
    m.trptract,
    m.trptrail,
    m.tot_trucks,
    m.tot_buses,
    m.tot_pwr,
    m.phy_str,
    ubci.phy_address,
    m.phy_city,
    ubci.phy_city,
    m.phy_st,
    ubci.phy_state,
    m.phy_zip,
    ubci.phy_zip_code,
    m.phy_natn,
    m.phy_cnty,
    ubci.phy_countyname,
    ubci.phy_latitude,
    ubci.phy_longitude,
    m.undeliv_phy,
    m.mai_str,
    ubci.mai_address,
    m.mai_city,
    ubci.mai_city,
    m.mai_st,
    ubci.mai_state,
    m.mai_zip,
    ubci.mai_zip_code,
    m.mai_natn,
    m.mai_cnty,
    ubci.mai_countyname,
    ubci.mai_latitude,
    ubci.mai_longitude,
    m.undeliv_mai,
    m.tel_num,
    m.fax_num,
    m.email_address,
    f.average_miles,
    m.tot_drs,
    m.cdl_drs,
    m.mcs150mileageyear,
    m.mlg150,
    m.ratedate,
    m.adddate,
    m.acc_rate,
    m.company_rep1,
    m.company_rep2,
    f.fleet_id,
    f.usdot,
    f.mcn,
    f.sic_code,
    f.update_date,
    f.active_flag,
    sub_a.commodities,
    f.company_type_code,
    f.product_code,
    f.company_name,

    li.phy_address1,
    li.phy_address2,
    li.phy_address3,
    li.phy_state,
    li.phy_city,
    li.phy_country,
    li.phy_zip,
    li.phy_countyfips,
    li.phy_latitude,
    li.phy_longitude,
    li.phy_location_id,
    li.mai_address1,
    li.mai_address2,
    li.mai_address3,
    li.mai_state,
    li.mai_city,
    li.mai_country,
    li.mai_zip,
    li.mai_countyfips,
    li.mai_latitude,
    li.mai_longitude,
    li.mai_location_id,

    c.first_name,
    c.middle_name,
    c.last_name,
    c.suffix,
    c.original_job_title,
    c.contact_id,
    sub_b.first_name, 
    sub_b.middle_name,
    sub_b.last_name,
    sub_b.suffix,
    sub_b.original_job_title,
    sub_b.contact_id,
    sub_c.phone_number,
    sub_c.phone_number_id,
    sub_c.tf_number,
    sub_c.tf_number_id,
    sub_c.fax_number,
    sub_c.fax_number_id,
    e.email_address,
    e.email_id,
    f.trucks_owned,
    f.trucks_leased,
    f.haz_trucks_owned,
    f.haz_trucks_leased,
    f.tractors_owned,
    f.tractors_leased,
    f.total_vehicles,
    f.trailers_owned,
    f.trailers_leased,
    f.haz_trailers_owned,
    f.haz_trailers_leased,
    sub_d.trailer_type_id,
    f.drivers,
    f.cdl_drivers,
    f.gvw_class_12,
    f.gvw_class_345,
    f.gvw_class_6,
    f.gvw_class_7,
    f.gvw_class_8,
    f.revenue,
    f.website,
    'Physical Address Change',
    ''

from fleetseek.fleet as f
join mcmis as m
on f.usdot = m.census_num
left join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
left join (
    select 
        fp.fleet_id,
        l1.address1 as phy_address1,
        l1.address2 as phy_address2,
        l1.address3 as phy_address3,
        l1.state_code as phy_state,
        l1.city as phy_city,
        l1.country_code as phy_country,
        l1.zip_code as phy_zip,
        l1.county_fips as phy_countyfips,
        l1.latitude as phy_latitude,
        l1.longitude as phy_longitude,
        l1.county_name as phy_county_name,
        l1.location_id as phy_location_id,
        l2.address1 as mai_address1,
        l2.address2 as mai_address2,
        l2.address3 as mai_address3,
        l2.state_code as mai_state,
        l2.city as mai_city,
        l2.country_code as mai_country,
        l2.zip_code as mai_zip,
        l2.county_fips as mai_countyfips,
        l2.latitude as mai_latitude,
        l2.longitude as mai_longitude,
        l2.county_name as mai_county_name,
        l2.location_id as mai_location_id

    from fleetseek.location as l1
    join fleetseek.fleet_primary as fp
    on l1.location_id = fp.primary_physical_location_id
    join fleetseek.location as l2
    on fp.primary_mailing_location_id = l2.location_id

) as li
on f.fleet_id = li.fleet_id
left join fleetseek.updates_bn_coder_input as ubci
on f.fleet_id = ubci.fs_fleet_id
left join fleetseek.contact as c
on fp.primary_contact_id = c.contact_id
left join(
    select 
        min(c.contact_id), 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id
    from fleetseek.fleet_contact as fc
    join fleetseek.fleet as f
    on fc.fleet_id = f.fleet_id
    join fleetseek.contact as c
    on fc.contact_id = c.contact_id
    where 
        sort_order = 2 and
        c.active = true 
    group by 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id

) as sub_b
on fp.fleet_id = sub_b.fleet_id
left join (
    select 
        fleet_id, 
        string_agg(commodity_id::varchar, ', ') as commodities
    from fleetseek.fleet_commodity
    group by fleet_id
) as sub_a
on f.fleet_id = sub_a.fleet_id
left join (
    select 
        pn.phone_number as phone_number, 
        pn.phone_number_id as phone_number_id, 
        pn2.phone_number as fax_number, 
        pn2.phone_number_id as fax_number_id,
        pn3.phone_number as tf_number, 
        pn3.phone_number_id as tf_number_id,
        fp.fleet_id
    from fleetseek.fleet_primary as fp
    left join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id 
    left join fleetseek.phone_number as pn2
    on fp.primary_fax_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_toll_free_id = pn3.phone_number_id
) as sub_c
on f.fleet_id = sub_c.fleet_id
left join fleetseek.email as e
on fp.primary_email_id = e.email_id
left join (
    select 
        fleet_id, 
        string_agg(trailer_type_id::varchar, ', ') as trailer_type_id
    from fleetseek.fleet_trailer_type
    group by fleet_id
) as sub_d
on f.fleet_id = sub_d.fleet_id
where 
    f.product_code in ('NMC', 'PFD') and
    f.active_flag = 'Y' and
    m.mcs150_date > f.update_date and
    m.mcs150_date > now() - interval '2 years' and
    f.total_vehicles > 20 and
    li.phy_state != m.phy_st 
    
    OR 
    
    (f.product_code in ('NMC', 'PFD') and
    f.active_flag = 'Y' and
    m.mcs150_date > f.update_date and
    m.mcs150_date > now() - interval '2 years' and
    f.total_vehicles >= 50 and
    upper(left(li.phy_address1, 3)) != left(m.phy_str, 3))
""")
conn.commit()

In [37]:
# Mailing address in different state for 20 or larger fleets and differing mailing address for fleets 50 or larger
cur.execute("""

insert into fleetseek.update_output 
select
    '',
    m.icc1,
    m.act_stat,
    m.carship,
    m.census_num,
    m.dbnum,
    m.rating,
    m.chngdate,
    m.mcs150_date,
    m.class,
    m.classdef,
    m.name,
    m.name_dba,
    m.crrinter,
    m.crrhmintra,
    m.crrintra,
    m.shpinter,
    m.shpintra,
    m.org,
    m.genfreight,
    m.household,
    m.metalsheet,
    m.motorveh,
    m.drivetow,
    m.logpole,
    m.bldgmat,
    m.mobilehome,
    m.machlrg,
    m.produce,
    m.oilfield,
    m.liqgas,
    m.intermodal,
    m.passengers,
    m.livestock,
    m.grainfeed,
    m.coalcoke,
    m.meat,
    m.garbage,
    m.usmail,
    m.chem,
    m.drybulk,
    m.coldfood,
    m.beverages,
    m.paperprod,
    m.utility,
    m.farmsupp,
    m.construct,
    m.waterwell,
    m.cargoothr,
    m.othercargo,
    m.hm_ind,
    m.owntruck,
    m.owntract,
    m.owntrail,
    m.trmtruck,
    m.trmtract,
    m.trmtrail,
    m.trptruck,
    m.trptract,
    m.trptrail,
    m.tot_trucks,
    m.tot_buses,
    m.tot_pwr,
    m.phy_str,
    ubci.phy_address,
    m.phy_city,
    ubci.phy_city,
    m.phy_st,
    ubci.phy_state,
    m.phy_zip,
    ubci.phy_zip_code,
    m.phy_natn,
    m.phy_cnty,
    ubci.phy_countyname,
    ubci.phy_latitude,
    ubci.phy_longitude,
    m.undeliv_phy,
    m.mai_str,
    ubci.mai_address,
    m.mai_city,
    ubci.mai_city,
    m.mai_st,
    ubci.mai_state,
    m.mai_zip,
    ubci.mai_zip_code,
    m.mai_natn,
    m.mai_cnty,
    ubci.mai_countyname,
    ubci.mai_latitude,
    ubci.mai_longitude,
    m.undeliv_mai,
    m.tel_num,
    m.fax_num,
    m.email_address,
    f.average_miles,
    m.tot_drs,
    m.cdl_drs,
    m.mcs150mileageyear,
    m.mlg150,
    m.ratedate,
    m.adddate,
    m.acc_rate,
    m.company_rep1,
    m.company_rep2,
    f.fleet_id,
    f.usdot,
    f.mcn,
    f.sic_code,
    f.update_date,
    f.active_flag,
    sub_a.commodities,
    f.company_type_code,
    f.product_code,
    f.company_name,

    li.phy_address1,
    li.phy_address2,
    li.phy_address3,
    li.phy_state,
    li.phy_city,
    li.phy_country,
    li.phy_zip,
    li.phy_countyfips,
    li.phy_latitude,
    li.phy_longitude,
    li.phy_location_id,
    li.mai_address1,
    li.mai_address2,
    li.mai_address3,
    li.mai_state,
    li.mai_city,
    li.mai_country,
    li.mai_zip,
    li.mai_countyfips,
    li.mai_latitude,
    li.mai_longitude,
    li.mai_location_id,

    c.first_name,
    c.middle_name,
    c.last_name,
    c.suffix,
    c.original_job_title,
    c.contact_id,
    sub_b.first_name, 
    sub_b.middle_name,
    sub_b.last_name,
    sub_b.suffix,
    sub_b.original_job_title,
    sub_b.contact_id,
    sub_c.phone_number,
    sub_c.phone_number_id,
    sub_c.tf_number,
    sub_c.tf_number_id,
    sub_c.fax_number,
    sub_c.fax_number_id,
    e.email_address,
    e.email_id,
    f.trucks_owned,
    f.trucks_leased,
    f.haz_trucks_owned,
    f.haz_trucks_leased,
    f.tractors_owned,
    f.tractors_leased,
    f.total_vehicles,
    f.trailers_owned,
    f.trailers_leased,
    f.haz_trailers_owned,
    f.haz_trailers_leased,
    sub_d.trailer_type_id,
    f.drivers,
    f.cdl_drivers,
    f.gvw_class_12,
    f.gvw_class_345,
    f.gvw_class_6,
    f.gvw_class_7,
    f.gvw_class_8,
    f.revenue,
    f.website,
    'Mailing Address Change',
    ''

from fleetseek.fleet as f
join mcmis as m
on f.usdot = m.census_num
left join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
left join (
    select 
        fp.fleet_id,
        l1.address1 as phy_address1,
        l1.address2 as phy_address2,
        l1.address3 as phy_address3,
        l1.state_code as phy_state,
        l1.city as phy_city,
        l1.country_code as phy_country,
        l1.zip_code as phy_zip,
        l1.county_fips as phy_countyfips,
        l1.latitude as phy_latitude,
        l1.longitude as phy_longitude,
        l1.county_name as phy_county_name,
        l1.location_id as phy_location_id,
        l2.address1 as mai_address1,
        l2.address2 as mai_address2,
        l2.address3 as mai_address3,
        l2.state_code as mai_state,
        l2.city as mai_city,
        l2.country_code as mai_country,
        l2.zip_code as mai_zip,
        l2.county_fips as mai_countyfips,
        l2.latitude as mai_latitude,
        l2.longitude as mai_longitude,
        l2.county_name as mai_county_name,
        l2.location_id as mai_location_id

    from fleetseek.location as l1
    join fleetseek.fleet_primary as fp
    on l1.location_id = fp.primary_physical_location_id
    join fleetseek.location as l2
    on fp.primary_mailing_location_id = l2.location_id

) as li
on f.fleet_id = li.fleet_id
left join fleetseek.updates_bn_coder_input as ubci
on f.fleet_id = ubci.fs_fleet_id
left join fleetseek.contact as c
on fp.primary_contact_id = c.contact_id
left join(
    select 
        min(c.contact_id), 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id
    from fleetseek.fleet_contact as fc
    join fleetseek.fleet as f
    on fc.fleet_id = f.fleet_id
    join fleetseek.contact as c
    on fc.contact_id = c.contact_id
    where 
        sort_order = 2 and
        c.active = true 
    group by 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id

) as sub_b
on fp.fleet_id = sub_b.fleet_id
left join (
    select 
        fleet_id, 
        string_agg(commodity_id::varchar, ', ') as commodities
    from fleetseek.fleet_commodity
    group by fleet_id
) as sub_a
on f.fleet_id = sub_a.fleet_id
left join (
    select 
        pn.phone_number as phone_number, 
        pn.phone_number_id as phone_number_id, 
        pn2.phone_number as fax_number, 
        pn2.phone_number_id as fax_number_id,
        pn3.phone_number as tf_number, 
        pn3.phone_number_id as tf_number_id,
        fp.fleet_id
    from fleetseek.fleet_primary as fp
    left join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id 
    left join fleetseek.phone_number as pn2
    on fp.primary_fax_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_toll_free_id = pn3.phone_number_id
) as sub_c
on f.fleet_id = sub_c.fleet_id
left join fleetseek.email as e
on fp.primary_email_id = e.email_id
left join (
    select 
        fleet_id, 
        string_agg(trailer_type_id::varchar, ', ') as trailer_type_id
    from fleetseek.fleet_trailer_type
    group by fleet_id
) as sub_d
on f.fleet_id = sub_d.fleet_id
where 
    f.product_code in ('NMC', 'PFD') and
    f.active_flag = 'Y' and
    m.mcs150_date > f.update_date and
    m.mcs150_date > now() - interval '2 years' and
    f.total_vehicles > 20 and
    li.mai_state != m.mai_st 
    
    OR
    
    (f.product_code in ('NMC', 'PFD') and
    f.active_flag = 'Y' and
    m.mcs150_date > f.update_date and
    m.mcs150_date > now() - interval '2 years' and
    f.total_vehicles >= 50 and
    upper(left(replace(li.mai_address1, ' ', ''), 3)) != left(replace(m.mai_str, ' ', ''), 3))
""")
conn.commit()

## Phone/Fax numbers

In [38]:
# 20+ fleet size with differing numbers with specific phone numbers to update
cur.execute("""
insert into fleetseek.update_output 
select
    '',
    m.icc1,
    m.act_stat,
    m.carship,
    m.census_num,
    m.dbnum,
    m.rating,
    m.chngdate,
    m.mcs150_date,
    m.class,
    m.classdef,
    m.name,
    m.name_dba,
    m.crrinter,
    m.crrhmintra,
    m.crrintra,
    m.shpinter,
    m.shpintra,
    m.org,
    m.genfreight,
    m.household,
    m.metalsheet,
    m.motorveh,
    m.drivetow,
    m.logpole,
    m.bldgmat,
    m.mobilehome,
    m.machlrg,
    m.produce,
    m.oilfield,
    m.liqgas,
    m.intermodal,
    m.passengers,
    m.livestock,
    m.grainfeed,
    m.coalcoke,
    m.meat,
    m.garbage,
    m.usmail,
    m.chem,
    m.drybulk,
    m.coldfood,
    m.beverages,
    m.paperprod,
    m.utility,
    m.farmsupp,
    m.construct,
    m.waterwell,
    m.cargoothr,
    m.othercargo,
    m.hm_ind,
    m.owntruck,
    m.owntract,
    m.owntrail,
    m.trmtruck,
    m.trmtract,
    m.trmtrail,
    m.trptruck,
    m.trptract,
    m.trptrail,
    m.tot_trucks,
    m.tot_buses,
    m.tot_pwr,
    m.phy_str,
    ubci.phy_address,
    m.phy_city,
    ubci.phy_city,
    m.phy_st,
    ubci.phy_state,
    m.phy_zip,
    ubci.phy_zip_code,
    m.phy_natn,
    m.phy_cnty,
    ubci.phy_countyname,
    ubci.phy_latitude,
    ubci.phy_longitude,
    m.undeliv_phy,
    m.mai_str,
    ubci.mai_address,
    m.mai_city,
    ubci.mai_city,
    m.mai_st,
    ubci.mai_state,
    m.mai_zip,
    ubci.mai_zip_code,
    m.mai_natn,
    m.mai_cnty,
    ubci.mai_countyname,
    ubci.mai_latitude,
    ubci.mai_longitude,
    m.undeliv_mai,
    m.tel_num,
    m.fax_num,
    m.email_address,
    f.average_miles,
    m.tot_drs,
    m.cdl_drs,
    m.mcs150mileageyear,
    m.mlg150,
    m.ratedate,
    m.adddate,
    m.acc_rate,
    m.company_rep1,
    m.company_rep2,
    f.fleet_id,
    f.usdot,
    f.mcn,
    f.sic_code,
    f.update_date,
    f.active_flag,
    sub_a.commodities,
    f.company_type_code,
    f.product_code,
    f.company_name,

    li.phy_address1,
    li.phy_address2,
    li.phy_address3,
    li.phy_state,
    li.phy_city,
    li.phy_country,
    li.phy_zip,
    li.phy_countyfips,
    li.phy_latitude,
    li.phy_longitude,
    li.phy_location_id,
    li.mai_address1,
    li.mai_address2,
    li.mai_address3,
    li.mai_state,
    li.mai_city,
    li.mai_country,
    li.mai_zip,
    li.mai_countyfips,
    li.mai_latitude,
    li.mai_longitude,
    li.mai_location_id,

    c.first_name,
    c.middle_name,
    c.last_name,
    c.suffix,
    c.original_job_title,
    c.contact_id,
    sub_b.first_name, 
    sub_b.middle_name,
    sub_b.last_name,
    sub_b.suffix,
    sub_b.original_job_title,
    sub_b.contact_id,
    sub_c.phone_number,
    sub_c.phone_number_id,
    sub_c.tf_number,
    sub_c.tf_number_id,
    sub_c.fax_number,
    sub_c.fax_number_id,
    e.email_address,
    e.email_id,
    f.trucks_owned,
    f.trucks_leased,
    f.haz_trucks_owned,
    f.haz_trucks_leased,
    f.tractors_owned,
    f.tractors_leased,
    f.total_vehicles,
    f.trailers_owned,
    f.trailers_leased,
    f.haz_trailers_owned,
    f.haz_trailers_leased,
    sub_d.trailer_type_id,
    f.drivers,
    f.cdl_drivers,
    f.gvw_class_12,
    f.gvw_class_345,
    f.gvw_class_6,
    f.gvw_class_7,
    f.gvw_class_8,
    f.revenue,
    f.website,
    case
        when (sub_c.phone_number != m.tel_num and (sub_c.fax_number = m.fax_num or m.fax_num is null) and m.tel_num not similar to ('888%|877%|866%|855%|844%|800%')) then 'Update Phone'
        when (sub_c.phone_number != m.tel_num and (sub_c.fax_number != m.fax_num or sub_c.fax_number is null) and m.tel_num not similar to ('888%|877%|866%|855%|844%|800%')) then 'Update Phone and Fax'
        when (sub_c.tf_number != m.tel_num and m.tel_num similar to ('888%|877%|866%|855%|844%|800%') and sub_c.fax_number != m.fax_num) then 'Update Fax and TF'
        when (sub_c.fax_number != m.fax_num) then 'Update Fax'
        when (sub_c.phone_number != m.tel_num and sub_c.tf_number != m.tel_num and m.tel_num similar to ('888%|877%|866%|855%|844%|800%')) then 'Update TF'
        when (sub_c.tf_number != m.tel_num and m.tel_num similar to ('888%|877%|866%|855%|844%|800%')) then 'Update TF'
    end,
    ''

from fleetseek.fleet as f
join mcmis as m
on f.usdot = m.census_num
left join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
left join (
    select 
        fp.fleet_id,
        l1.address1 as phy_address1,
        l1.address2 as phy_address2,
        l1.address3 as phy_address3,
        l1.state_code as phy_state,
        l1.city as phy_city,
        l1.country_code as phy_country,
        l1.zip_code as phy_zip,
        l1.county_fips as phy_countyfips,
        l1.latitude as phy_latitude,
        l1.longitude as phy_longitude,
        l1.location_id as phy_location_id,
        l2.address1 as mai_address1,
        l2.address2 as mai_address2,
        l2.address3 as mai_address3,
        l2.state_code as mai_state,
        l2.city as mai_city,
        l2.country_code as mai_country,
        l2.zip_code as mai_zip,
        l2.county_fips as mai_countyfips,
        l2.latitude as mai_latitude,
        l2.longitude as mai_longitude,
        l2.location_id as mai_location_id

    from fleetseek.location as l1
    join fleetseek.fleet_primary as fp
    on l1.location_id = fp.primary_physical_location_id
    join fleetseek.location as l2
    on fp.primary_mailing_location_id = l2.location_id

) as li
on f.fleet_id = li.fleet_id
left join fleetseek.updates_bn_coder_input as ubci
on f.fleet_id = ubci.fs_fleet_id
left join fleetseek.contact as c
on fp.primary_contact_id = c.contact_id
left join(
    select 
        min(c.contact_id), 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id
    from fleetseek.fleet_contact as fc
    join fleetseek.fleet as f
    on fc.fleet_id = f.fleet_id
    join fleetseek.contact as c
    on fc.contact_id = c.contact_id
    where 
        sort_order = 2 and
        c.active = true 
    group by 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id

) as sub_b
on fp.fleet_id = sub_b.fleet_id
left join (
    select 
        fleet_id, 
        string_agg(commodity_id::varchar, ', ') as commodities
    from fleetseek.fleet_commodity
    group by fleet_id
) as sub_a
on f.fleet_id = sub_a.fleet_id
left join (
    select 
        pn.phone_number as phone_number, 
        pn.phone_number_id as phone_number_id, 
        pn2.phone_number as fax_number, 
        pn2.phone_number_id as fax_number_id,
        pn3.phone_number as tf_number, 
        pn3.phone_number_id as tf_number_id,
        fp.fleet_id
    from fleetseek.fleet_primary as fp
    left join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id 
    left join fleetseek.phone_number as pn2
    on fp.primary_fax_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_toll_free_id = pn3.phone_number_id
) as sub_c
on f.fleet_id = sub_c.fleet_id
left join fleetseek.email as e
on fp.primary_email_id = e.email_id
left join (
    select 
        fleet_id, 
        string_agg(trailer_type_id::varchar, ', ') as trailer_type_id
    from fleetseek.fleet_trailer_type
    group by fleet_id
) as sub_d
on f.fleet_id = sub_d.fleet_id

where 
    f.product_code in ('NMC', 'PFD') and
    f.active_flag = 'Y' and
    m.mcs150_date > f.update_date and
    m.mcs150_date > now() - interval '2 years' and
    f.total_vehicles > 20 and
    ((left(sub_c.phone_number, 3) != left(m.tel_num, 3) and m.tel_num not similar to ('888%|877%|866%|855%|844%|800%')) or (left(sub_c.fax_number, 3) != left(m.fax_num, 3)) or (sub_c.tf_number != m.tel_num and m.tel_num similar to ('888%|877%|866%|855%|844%|800%')))

""")
conn.commit()

In [39]:
# Any fleet size with phone numbers that start with 0 or 1, 
cur.execute("""
insert into fleetseek.update_output 
select
    '',
    m.icc1,
    m.act_stat,
    m.carship,
    m.census_num,
    m.dbnum,
    m.rating,
    m.chngdate,
    m.mcs150_date,
    m.class,
    m.classdef,
    m.name,
    m.name_dba,
    m.crrinter,
    m.crrhmintra,
    m.crrintra,
    m.shpinter,
    m.shpintra,
    m.org,
    m.genfreight,
    m.household,
    m.metalsheet,
    m.motorveh,
    m.drivetow,
    m.logpole,
    m.bldgmat,
    m.mobilehome,
    m.machlrg,
    m.produce,
    m.oilfield,
    m.liqgas,
    m.intermodal,
    m.passengers,
    m.livestock,
    m.grainfeed,
    m.coalcoke,
    m.meat,
    m.garbage,
    m.usmail,
    m.chem,
    m.drybulk,
    m.coldfood,
    m.beverages,
    m.paperprod,
    m.utility,
    m.farmsupp,
    m.construct,
    m.waterwell,
    m.cargoothr,
    m.othercargo,
    m.hm_ind,
    m.owntruck,
    m.owntract,
    m.owntrail,
    m.trmtruck,
    m.trmtract,
    m.trmtrail,
    m.trptruck,
    m.trptract,
    m.trptrail,
    m.tot_trucks,
    m.tot_buses,
    m.tot_pwr,
    m.phy_str,
    ubci.phy_address,
    m.phy_city,
    ubci.phy_city,
    m.phy_st,
    ubci.phy_state,
    m.phy_zip,
    ubci.phy_zip_code,
    m.phy_natn,
    m.phy_cnty,
    ubci.phy_countyname,
    ubci.phy_latitude,
    ubci.phy_longitude,
    m.undeliv_phy,
    m.mai_str,
    ubci.mai_address,
    m.mai_city,
    ubci.mai_city,
    m.mai_st,
    ubci.mai_state,
    m.mai_zip,
    ubci.mai_zip_code,
    m.mai_natn,
    m.mai_cnty,
    ubci.mai_countyname,
    ubci.mai_latitude,
    ubci.mai_longitude,
    m.undeliv_mai,
    m.tel_num,
    m.fax_num,
    m.email_address,
    f.average_miles,
    m.tot_drs,
    m.cdl_drs,
    m.mcs150mileageyear,
    m.mlg150,
    m.ratedate,
    m.adddate,
    m.acc_rate,
    m.company_rep1,
    m.company_rep2,
    f.fleet_id,
    f.usdot,
    f.mcn,
    f.sic_code,
    f.update_date,
    f.active_flag,
    sub_a.commodities,
    f.company_type_code,
    f.product_code,
    f.company_name,

    li.phy_address1,
    li.phy_address2,
    li.phy_address3,
    li.phy_state,
    li.phy_city,
    li.phy_country,
    li.phy_zip,
    li.phy_countyfips,
    li.phy_latitude,
    li.phy_longitude,
    li.phy_location_id,
    li.mai_address1,
    li.mai_address2,
    li.mai_address3,
    li.mai_state,
    li.mai_city,
    li.mai_country,
    li.mai_zip,
    li.mai_countyfips,
    li.mai_latitude,
    li.mai_longitude,
    li.mai_location_id,

    c.first_name,
    c.middle_name,
    c.last_name,
    c.suffix,
    c.original_job_title,
    c.contact_id,
    sub_b.first_name, 
    sub_b.middle_name,
    sub_b.last_name,
    sub_b.suffix,
    sub_b.original_job_title,
    sub_b.contact_id,
    sub_c.phone_number,
    sub_c.phone_number_id,
    sub_c.tf_number,
    sub_c.tf_number_id,
    sub_c.fax_number,
    sub_c.fax_number_id,
    e.email_address,
    e.email_id,
    f.trucks_owned,
    f.trucks_leased,
    f.haz_trucks_owned,
    f.haz_trucks_leased,
    f.tractors_owned,
    f.tractors_leased,
    f.total_vehicles,
    f.trailers_owned,
    f.trailers_leased,
    f.haz_trailers_owned,
    f.haz_trailers_leased,
    sub_d.trailer_type_id,
    f.drivers,
    f.cdl_drivers,
    f.gvw_class_12,
    f.gvw_class_345,
    f.gvw_class_6,
    f.gvw_class_7,
    f.gvw_class_8,
    f.revenue,
    f.website,
    'Phone Number Errors'
    ''

from fleetseek.fleet as f
join mcmis as m
on f.usdot = m.census_num
left join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
left join (
    select 
        fp.fleet_id,
        l1.address1 as phy_address1,
        l1.address2 as phy_address2,
        l1.address3 as phy_address3,
        l1.state_code as phy_state,
        l1.city as phy_city,
        l1.country_code as phy_country,
        l1.zip_code as phy_zip,
        l1.county_fips as phy_countyfips,
        l1.latitude as phy_latitude,
        l1.longitude as phy_longitude,
        l1.location_id as phy_location_id,
        l2.address1 as mai_address1,
        l2.address2 as mai_address2,
        l2.address3 as mai_address3,
        l2.state_code as mai_state,
        l2.city as mai_city,
        l2.country_code as mai_country,
        l2.zip_code as mai_zip,
        l2.county_fips as mai_countyfips,
        l2.latitude as mai_latitude,
        l2.longitude as mai_longitude,
        l2.location_id as mai_location_id

    from fleetseek.location as l1
    join fleetseek.fleet_primary as fp
    on l1.location_id = fp.primary_physical_location_id
    join fleetseek.location as l2
    on fp.primary_mailing_location_id = l2.location_id

) as li
on f.fleet_id = li.fleet_id
left join fleetseek.updates_bn_coder_input as ubci
on f.fleet_id = ubci.fs_fleet_id
left join fleetseek.contact as c
on fp.primary_contact_id = c.contact_id
left join(
    select 
        min(c.contact_id), 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id
    from fleetseek.fleet_contact as fc
    join fleetseek.fleet as f
    on fc.fleet_id = f.fleet_id
    join fleetseek.contact as c
    on fc.contact_id = c.contact_id
    where 
        sort_order = 2 and
        c.active = true 
    group by 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id

) as sub_b
on fp.fleet_id = sub_b.fleet_id
left join (
    select 
        fleet_id, 
        string_agg(commodity_id::varchar, ', ') as commodities
    from fleetseek.fleet_commodity
    group by fleet_id
) as sub_a
on f.fleet_id = sub_a.fleet_id
left join (
    select 
        pn.phone_number as phone_number, 
        pn.phone_number_id as phone_number_id, 
        pn2.phone_number as fax_number, 
        pn2.phone_number_id as fax_number_id,
        pn3.phone_number as tf_number, 
        pn3.phone_number_id as tf_number_id,
        fp.fleet_id
    from fleetseek.fleet_primary as fp
    left join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id 
    left join fleetseek.phone_number as pn2
    on fp.primary_fax_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_toll_free_id = pn3.phone_number_id
) as sub_c
on f.fleet_id = sub_c.fleet_id
left join fleetseek.email as e
on fp.primary_email_id = e.email_id
left join (
    select 
        fleet_id, 
        string_agg(trailer_type_id::varchar, ', ') as trailer_type_id
    from fleetseek.fleet_trailer_type
    group by fleet_id
) as sub_d
on f.fleet_id = sub_d.fleet_id

where 
    f.product_code in ('NMC', 'PFD', 'OOD') and
    f.active_flag = 'Y' and
    m.mcs150_date > f.update_date and
    m.mcs150_date > now() - interval '2 years' and
    ((left(m.tel_num, 1) = '0' or left(m.tel_num, 1) = '1') or
    length(m.tel_num) <> 10)
    
""")
conn.commit()

## Fleet totals

In [40]:
# Discrepancies for fleets less than 50
# If a fleet with less than 50 has an increase of 50 or more it will go to an analyst for review
# Doesn't include a where clause for fleet type since it will include all fleet types
cur.execute("""
insert into fleetseek.update_output 

select
    '',
    m.icc1,
    m.act_stat,
    m.carship,
    m.census_num,
    m.dbnum,
    m.rating,
    m.chngdate,
    m.mcs150_date,
    m.class,
    m.classdef,
    m.name,
    m.name_dba,
    m.crrinter,
    m.crrhmintra,
    m.crrintra,
    m.shpinter,
    m.shpintra,
    m.org,
    m.genfreight,
    m.household,
    m.metalsheet,
    m.motorveh,
    m.drivetow,
    m.logpole,
    m.bldgmat,
    m.mobilehome,
    m.machlrg,
    m.produce,
    m.oilfield,
    m.liqgas,
    m.intermodal,
    m.passengers,
    m.livestock,
    m.grainfeed,
    m.coalcoke,
    m.meat,
    m.garbage,
    m.usmail,
    m.chem,
    m.drybulk,
    m.coldfood,
    m.beverages,
    m.paperprod,
    m.utility,
    m.farmsupp,
    m.construct,
    m.waterwell,
    m.cargoothr,
    m.othercargo,
    m.hm_ind,
    m.owntruck,
    m.owntract,
    m.owntrail,
    m.trmtruck,
    m.trmtract,
    m.trmtrail,
    m.trptruck,
    m.trptract,
    m.trptrail,
    m.tot_trucks,
    m.tot_buses,
    m.tot_pwr,
    m.phy_str,
    ubci.phy_address,
    m.phy_city,
    ubci.phy_city,
    m.phy_st,
    ubci.phy_state,
    m.phy_zip,
    ubci.phy_zip_code,
    m.phy_natn,
    m.phy_cnty,
    ubci.phy_countyname,
    ubci.phy_latitude,
    ubci.phy_longitude,
    m.undeliv_phy,
    m.mai_str,
    ubci.mai_address,
    m.mai_city,
    ubci.mai_city,
    m.mai_st,
    ubci.mai_state,
    m.mai_zip,
    ubci.mai_zip_code,
    m.mai_natn,
    m.mai_cnty,
    ubci.mai_countyname,
    ubci.mai_latitude,
    ubci.mai_longitude,
    m.undeliv_mai,
    m.tel_num,
    m.fax_num,
    m.email_address,
    f.average_miles,
    m.tot_drs,
    m.cdl_drs,
    m.mcs150mileageyear,
    m.mlg150,
    m.ratedate,
    m.adddate,
    m.acc_rate,
    m.company_rep1,
    m.company_rep2,
    f.fleet_id,
    f.usdot,
    f.mcn,
    f.sic_code,
    f.update_date,
    f.active_flag,
    sub_a.commodities,
    f.company_type_code,
    f.product_code,
    f.company_name,

    li.phy_address1,
    li.phy_address2,
    li.phy_address3,
    li.phy_state,
    li.phy_city,
    li.phy_country,
    li.phy_zip,
    li.phy_countyfips,
    li.phy_latitude,
    li.phy_longitude,
    li.phy_location_id,
    li.mai_address1,
    li.mai_address2,
    li.mai_address3,
    li.mai_state,
    li.mai_city,
    li.mai_country,
    li.mai_zip,
    li.mai_countyfips,
    li.mai_latitude,
    li.mai_longitude,
    li.mai_location_id,

    c.first_name,
    c.middle_name,
    c.last_name,
    c.suffix,
    c.original_job_title,
    c.contact_id,
    sub_b.first_name, 
    sub_b.middle_name,
    sub_b.last_name,
    sub_b.suffix,
    sub_b.original_job_title,
    sub_b.contact_id,
    sub_c.phone_number,
    sub_c.phone_number_id,
    sub_c.tf_number,
    sub_c.tf_number_id,
    sub_c.fax_number,
    sub_c.fax_number_id,
    e.email_address,
    e.email_id,
    f.trucks_owned,
    f.trucks_leased,
    f.haz_trucks_owned,
    f.haz_trucks_leased,
    f.tractors_owned,
    f.tractors_leased,
    f.total_vehicles,
    f.trailers_owned,
    f.trailers_leased,
    f.haz_trailers_owned,
    f.haz_trailers_leased,
    sub_d.trailer_type_id,
    f.drivers,
    f.cdl_drivers,
    f.gvw_class_12,
    f.gvw_class_345,
    f.gvw_class_6,
    f.gvw_class_7,
    f.gvw_class_8,
    f.revenue,
    f.website,
    'Fleet Discrepancy (Fleet smaller than 50)',
    ''

from fleetseek.fleet as f
join mcmis as m
on f.usdot = m.census_num
join (
    select 
        f.fleet_id, 
        f.total_vehicles,
        f.product_code,
        f.active_flag,
        m.mcs150_date,
        f.update_date,

        sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract) as mcmis_total_vehicles
    from fleetseek.fleet as f
    join mcmis as m
    on f.usdot = m.census_num
    where

        f.active_flag = 'Y' and
        m.mcs150_date > f.update_date and
        m.mcs150_date > now() - interval '2 years' and
        f.total_vehicles < 50 
    group by 
        f.fleet_id, 
        f.total_vehicles,
        f.product_code,
        f.active_flag,
        m.mcs150_date,
        f.update_date,
        m.owntruck,
        m.trmtruck,
        m.trptruck,
        m.owntract,
        m.trmtract,
        m.trptract
    having abs(f.total_vehicles - sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract)) >= 50
) as sub_e
on f.fleet_id = sub_e.fleet_id
left join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
left join (
    select 
        fp.fleet_id,
        l1.address1 as phy_address1,
        l1.address2 as phy_address2,
        l1.address3 as phy_address3,
        l1.state_code as phy_state,
        l1.city as phy_city,
        l1.country_code as phy_country,
        l1.zip_code as phy_zip,
        l1.county_fips as phy_countyfips,
        l1.latitude as phy_latitude,
        l1.longitude as phy_longitude,
        l1.location_id as phy_location_id,
        l2.address1 as mai_address1,
        l2.address2 as mai_address2,
        l2.address3 as mai_address3,
        l2.state_code as mai_state,
        l2.city as mai_city,
        l2.country_code as mai_country,
        l2.zip_code as mai_zip,
        l2.county_fips as mai_countyfips,
        l2.latitude as mai_latitude,
        l2.longitude as mai_longitude,
        l2.location_id as mai_location_id

    from fleetseek.location as l1
    join fleetseek.fleet_primary as fp
    on l1.location_id = fp.primary_physical_location_id
    join fleetseek.location as l2
    on fp.primary_mailing_location_id = l2.location_id

) as li
on f.fleet_id = li.fleet_id
left join fleetseek.contact as c
on fp.primary_contact_id = c.contact_id
left join (
    select 
        fleet_id, 
        string_agg(commodity_id::varchar, ', ') as commodities
    from fleetseek.fleet_commodity
    group by fleet_id
) as sub_a
on f.fleet_id = sub_a.fleet_id
left join (
    select 
        pn.phone_number as phone_number, 
        pn.phone_number_id as phone_number_id, 
        pn2.phone_number as fax_number, 
        pn2.phone_number_id as fax_number_id,
        pn3.phone_number as tf_number, 
        pn3.phone_number_id as tf_number_id,
        fp.fleet_id
    from fleetseek.fleet_primary as fp
    left join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id 
    left join fleetseek.phone_number as pn2
    on fp.primary_fax_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_toll_free_id = pn3.phone_number_id
) as sub_c
on f.fleet_id = sub_c.fleet_id
left join fleetseek.updates_bn_coder_input as ubci
on f.fleet_id = ubci.fs_fleet_id
left join(
    select 
        min(c.contact_id), 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id
    from fleetseek.fleet_contact as fc
    join fleetseek.fleet as f
    on fc.fleet_id = f.fleet_id
    join fleetseek.contact as c
    on fc.contact_id = c.contact_id
    where 
        sort_order = 2 and
        c.active = true 
    group by 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id	
) as sub_b
on fp.fleet_id = sub_b.fleet_id
left join fleetseek.email as e
on fp.primary_email_id = e.email_id
left join (
    select 
        fleet_id, 
        string_agg(trailer_type_id::varchar, ', ') as trailer_type_id
    from fleetseek.fleet_trailer_type
    group by fleet_id
) as sub_d
on f.fleet_id = sub_d.fleet_id
""")
conn.commit()

In [41]:
# Discrepancies for fleets that are larger 50
# Will be based on a difference of 20% or more
cur.execute("""
insert into fleetseek.update_output 
select
    '',
    m.icc1,
    m.act_stat,
    m.carship,
    m.census_num,
    m.dbnum,
    m.rating,
    m.chngdate,
    m.mcs150_date,
    m.class,
    m.classdef,
    m.name,
    m.name_dba,
    m.crrinter,
    m.crrhmintra,
    m.crrintra,
    m.shpinter,
    m.shpintra,
    m.org,
    m.genfreight,
    m.household,
    m.metalsheet,
    m.motorveh,
    m.drivetow,
    m.logpole,
    m.bldgmat,
    m.mobilehome,
    m.machlrg,
    m.produce,
    m.oilfield,
    m.liqgas,
    m.intermodal,
    m.passengers,
    m.livestock,
    m.grainfeed,
    m.coalcoke,
    m.meat,
    m.garbage,
    m.usmail,
    m.chem,
    m.drybulk,
    m.coldfood,
    m.beverages,
    m.paperprod,
    m.utility,
    m.farmsupp,
    m.construct,
    m.waterwell,
    m.cargoothr,
    m.othercargo,
    m.hm_ind,
    m.owntruck,
    m.owntract,
    m.owntrail,
    m.trmtruck,
    m.trmtract,
    m.trmtrail,
    m.trptruck,
    m.trptract,
    m.trptrail,
    m.tot_trucks,
    m.tot_buses,
    m.tot_pwr,
    m.phy_str,
    ubci.phy_address,
    m.phy_city,
    ubci.phy_city,
    m.phy_st,
    ubci.phy_state,
    m.phy_zip,
    ubci.phy_zip_code,
    m.phy_natn,
    m.phy_cnty,
    ubci.phy_countyname,
    ubci.phy_latitude,
    ubci.phy_longitude,
    m.undeliv_phy,
    m.mai_str,
    ubci.mai_address,
    m.mai_city,
    ubci.mai_city,
    m.mai_st,
    ubci.mai_state,
    m.mai_zip,
    ubci.mai_zip_code,
    m.mai_natn,
    m.mai_cnty,
    ubci.mai_countyname,
    ubci.mai_latitude,
    ubci.mai_longitude,
    m.undeliv_mai,
    m.tel_num,
    m.fax_num,
    m.email_address,
    f.average_miles,
    m.tot_drs,
    m.cdl_drs,
    m.mcs150mileageyear,
    m.mlg150,
    m.ratedate,
    m.adddate,
    m.acc_rate,
    m.company_rep1,
    m.company_rep2,
    f.fleet_id,
    f.usdot,
    f.mcn,
    f.sic_code,
    f.update_date,
    f.active_flag,
    sub_a.commodities,
    f.company_type_code,
    f.product_code,
    f.company_name,

    li.phy_address1,
    li.phy_address2,
    li.phy_address3,
    li.phy_state,
    li.phy_city,
    li.phy_country,
    li.phy_zip,
    li.phy_countyfips,
    li.phy_latitude,
    li.phy_longitude,
    li.phy_location_id,
    li.mai_address1,
    li.mai_address2,
    li.mai_address3,
    li.mai_state,
    li.mai_city,
    li.mai_country,
    li.mai_zip,
    li.mai_countyfips,
    li.mai_latitude,
    li.mai_longitude,
    li.mai_location_id,

    c.first_name,
    c.middle_name,
    c.last_name,
    c.suffix,
    c.original_job_title,
    c.contact_id,
    sub_b.first_name, 
    sub_b.middle_name,
    sub_b.last_name,
    sub_b.suffix,
    sub_b.original_job_title,
    sub_b.contact_id,
    sub_c.phone_number,
    sub_c.phone_number_id,
    sub_c.tf_number,
    sub_c.tf_number_id,
    sub_c.fax_number,
    sub_c.fax_number_id,
    e.email_address,
    e.email_id,
    f.trucks_owned,
    f.trucks_leased,
    f.haz_trucks_owned,
    f.haz_trucks_leased,
    f.tractors_owned,
    f.tractors_leased,
    f.total_vehicles,
    f.trailers_owned,
    f.trailers_leased,
    f.haz_trailers_owned,
    f.haz_trailers_leased,
    sub_d.trailer_type_id,
    f.drivers,
    f.cdl_drivers,
    f.gvw_class_12,
    f.gvw_class_345,
    f.gvw_class_6,
    f.gvw_class_7,
    f.gvw_class_8,
    f.revenue,
    f.website,
    'Fleet Discrepancy (Fleet larger than 50)',
    ''

from fleetseek.fleet as f
join mcmis as m
on f.usdot = m.census_num
join(
    select 
        sub_q.fleet_id, 
        sub_q.active_flag,
        sub_q.update_date,
        sub_q.mcs150_date,
        sub_q.fs_total_vehicles,
        sub_q.mcmis_total_vehicles,
        sub_q.absolute_difference,
        sub_q.average_of_two,
        round(sub_q.absolute_difference / nullif(sub_q.average_of_two, 0) * 100, 2) as pct_difference

    from(
        select 
            f.fleet_id,
            f.active_flag,
            f.update_date,
            m.mcs150_date,
            f.total_vehicles as fs_total_vehicles, 
            sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract) as mcmis_total_vehicles,
            abs(f.total_vehicles - sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract)) as absolute_difference,
            round((sum(f.total_vehicles + m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract)) / 2::numeric, 2) as average_of_two

        from fleetseek.fleet as f
        join mcmis as m
        on f.usdot = m.census_num
        where 
            active_flag = 'Y' and
            m.mcs150_date > f.update_date and
            m.mcs150_date > now() - interval '2 years'
        group by 
            f.total_vehicles, 
            m.owntruck, 
            m.trmtruck, 
            m.trptruck, 
            m.owntract, 
            m.trmtract, 
            m.trptract,
            f.fleet_id,
            f.active_flag,
            f.update_date,
            m.mcs150_date

    ) as sub_q
    join fleetseek.fleet as f
    on sub_q.fleet_id = f.fleet_id
    where 
        fs_total_vehicles > 50 and
        round(sub_q.absolute_difference / nullif(sub_q.average_of_two, 0) * 100, 2) >= 20

    group by 
        sub_q.mcmis_total_vehicles, 
        sub_q.fs_total_vehicles, 
        sub_q.absolute_difference, 
        sub_q.average_of_two,
        sub_q.fleet_id, 
        sub_q.active_flag,
        sub_q.update_date,
        sub_q.mcs150_date

) as sub_e
on f.fleet_id = sub_e.fleet_id
left join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
left join (
    select 
        fp.fleet_id,
        l1.address1 as phy_address1,
        l1.address2 as phy_address2,
        l1.address3 as phy_address3,
        l1.state_code as phy_state,
        l1.city as phy_city,
        l1.country_code as phy_country,
        l1.zip_code as phy_zip,
        l1.county_fips as phy_countyfips,
        l1.latitude as phy_latitude,
        l1.longitude as phy_longitude,
        l1.location_id as phy_location_id,
        l2.address1 as mai_address1,
        l2.address2 as mai_address2,
        l2.address3 as mai_address3,
        l2.state_code as mai_state,
        l2.city as mai_city,
        l2.country_code as mai_country,
        l2.zip_code as mai_zip,
        l2.county_fips as mai_countyfips,
        l2.latitude as mai_latitude,
        l2.longitude as mai_longitude,
        l2.location_id as mai_location_id

    from fleetseek.location as l1
    join fleetseek.fleet_primary as fp
    on l1.location_id = fp.primary_physical_location_id
    join fleetseek.location as l2
    on fp.primary_mailing_location_id = l2.location_id

) as li
on f.fleet_id = li.fleet_id
left join fleetseek.contact as c
on fp.primary_contact_id = c.contact_id
left join (
    select fleet_id, string_agg(commodity_id::varchar, ', ') as commodities
    from fleetseek.fleet_commodity
    group by fleet_id
) as sub_a
on f.fleet_id = sub_a.fleet_id
left join (
    select 
        pn.phone_number as phone_number, 
        pn.phone_number_id as phone_number_id, 
        pn2.phone_number as fax_number, 
        pn2.phone_number_id as fax_number_id,
        pn3.phone_number as tf_number, 
        pn3.phone_number_id as tf_number_id,
        fp.fleet_id
    from fleetseek.fleet_primary as fp
    join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id 
    left join fleetseek.phone_number as pn2
    on fp.primary_fax_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_toll_free_id = pn3.phone_number_id
) as sub_c
on f.fleet_id = sub_c.fleet_id
left join fleetseek.updates_bn_coder_input as ubci
on f.fleet_id = ubci.fs_fleet_id
left join(
    select 
        min(c.contact_id), 
    
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id
    from fleetseek.fleet_contact as fc
    join fleetseek.fleet as f
    on fc.fleet_id = f.fleet_id
    join fleetseek.contact as c
    on fc.contact_id = c.contact_id
    where 
        sort_order = 2 and
        c.active = true 
    group by 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id	
) as sub_b
on fp.fleet_id = sub_b.fleet_id
left join fleetseek.email as e
on fp.primary_email_id = e.email_id
left join (
    select 
        fleet_id, 
        string_agg(trailer_type_id::varchar, ', ') as trailer_type_id
    from fleetseek.fleet_trailer_type
    group by fleet_id
) as sub_d
on f.fleet_id = sub_d.fleet_id

""")
conn.commit()

## Trailer differences

In [ ]:
cur.execute("""
insert into fleetseek.update_output
select
    '',
    m.icc1,
    m.act_stat,
    m.carship,
    m.census_num,
    m.dbnum,
    m.rating,
    m.chngdate,
    m.mcs150_date,
    m.class,
    m.classdef,
    m.name,
    m.name_dba,
    m.crrinter,
    m.crrhmintra,
    m.crrintra,
    m.shpinter,
    m.shpintra,
    m.org,
    m.genfreight,
    m.household,
    m.metalsheet,
    m.motorveh,
    m.drivetow,
    m.logpole,
    m.bldgmat,
    m.mobilehome,
    m.machlrg,
    m.produce,
    m.oilfield,
    m.liqgas,
    m.intermodal,
    m.passengers,
    m.livestock,
    m.grainfeed,
    m.coalcoke,
    m.meat,
    m.garbage,
    m.usmail,
    m.chem,
    m.drybulk,
    m.coldfood,
    m.beverages,
    m.paperprod,
    m.utility,
    m.farmsupp,
    m.construct,
    m.waterwell,
    m.cargoothr,
    m.othercargo,
    m.hm_ind,
    m.owntruck,
    m.owntract,
    m.owntrail,
    m.trmtruck,
    m.trmtract,
    m.trmtrail,
    m.trptruck,
    m.trptract,
    m.trptrail,
    m.tot_trucks,
    m.tot_buses,
    m.tot_pwr,
    m.phy_str,
    ubci.phy_address,
    m.phy_city,
    ubci.phy_city,
    m.phy_st,
    ubci.phy_state,
    m.phy_zip,
    ubci.phy_zip_code,
    m.phy_natn,
    m.phy_cnty,
    ubci.phy_countyname,
    ubci.phy_latitude,
    ubci.phy_longitude,
    m.undeliv_phy,
    m.mai_str,
    ubci.mai_address,
    m.mai_city,
    ubci.mai_city,
    m.mai_st,
    ubci.mai_state,
    m.mai_zip,
    ubci.mai_zip_code,
    m.mai_natn,
    m.mai_cnty,
    ubci.mai_countyname,
    ubci.mai_latitude,
    ubci.mai_longitude,
    m.undeliv_mai,
    m.tel_num,
    m.fax_num,
    m.email_address,
    f.average_miles,
    m.tot_drs,
    m.cdl_drs,
    m.mcs150mileageyear,
    m.mlg150,
    m.ratedate,
    m.adddate,
    m.acc_rate,
    m.company_rep1,
    m.company_rep2,
    f.fleet_id,
    f.usdot,
    f.mcn,
    f.sic_code,
    f.update_date,
    f.active_flag,
    sub_a.commodities,
    f.company_type_code,
    f.product_code,
    f.company_name,

    li.phy_address1,
    li.phy_address2,
    li.phy_address3,
    li.phy_state,
    li.phy_city,
    li.phy_country,
    li.phy_zip,
    li.phy_countyfips,
    li.phy_latitude,
    li.phy_longitude,
    li.phy_location_id,
    li.mai_address1,
    li.mai_address2,
    li.mai_address3,
    li.mai_state,
    li.mai_city,
    li.mai_country,
    li.mai_zip,
    li.mai_countyfips,
    li.mai_latitude,
    li.mai_longitude,
    li.mai_location_id,

    c.first_name,
    c.middle_name,
    c.last_name,
    c.suffix,
    c.original_job_title,
    c.contact_id,
    sub_b.first_name, 
    sub_b.middle_name,
    sub_b.last_name,
    sub_b.suffix,
    sub_b.original_job_title,
    sub_b.contact_id,
    sub_c.phone_number,
    sub_c.phone_number_id,
    sub_c.tf_number,
    sub_c.tf_number_id,
    sub_c.fax_number,
    sub_c.fax_number_id,
    e.email_address,
    e.email_id,
    f.trucks_owned,
    f.trucks_leased,
    f.haz_trucks_owned,
    f.haz_trucks_leased,
    f.tractors_owned,
    f.tractors_leased,
    f.total_vehicles,
    f.trailers_owned,
    f.trailers_leased,
    f.haz_trailers_owned,
    f.haz_trailers_leased,
    sub_d.trailer_type_id,
    f.drivers,
    f.cdl_drivers,
    f.gvw_class_12,
    f.gvw_class_345,
    f.gvw_class_6,
    f.gvw_class_7,
    f.gvw_class_8,
    f.revenue,
    f.website,
    'Trailer Discrepancy',
    ''

from fleetseek.fleet as f
join (
    select 
        f.fleet_id, 
        f.dot_hmind,
        f.usdot,
        f.total_vehicles,
        f.total_trailers,
        f.product_code,
        f.active_flag,
        m.mcs150_date,
        f.update_date,
        sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract) as mcmis_total_vehicles,
        sum(m.owntrail + m.trmtrail + m.trptrail) as mcmis_total_trailers,
        abs(f.total_trailers - sum(m.owntrail + m.trmtrail + m.trptrail)) as absolute_trailer_difference,
        abs(f.total_vehicles - sum(m.owntruck + m.trmtruck + m.trptruck + m.owntract + m.trmtract + m.trptract)) as absolute_vehicle_difference

    from fleetseek.fleet as f
    join my_public.mcmis as m
    on f.usdot = m.census_num
    where
        f.active_flag = 'Y' and
        m.mcs150_date > f.update_date and
        m.mcs150_date > now() - interval '2 years' and
        f.total_vehicles < 50 
    group by 
        f.fleet_id, 
        f.total_vehicles,
        f.product_code,
        f.active_flag,
        m.mcs150_date,
        f.update_date,
        m.owntruck,
        m.trmtruck,
        m.trptruck,
        m.owntract,
        m.trmtract,
        m.trptract,
        m.owntrail,
        m.trmtrail,
        m.trptrail,
        f.dot_hmind,
        f.usdot

    having abs(f.total_trailers - sum(m.owntrail + m.trmtrail + m.trptrail)) >= 100
) as sub_e
on f.fleet_id = sub_e.fleet_id
join my_public.mcmis as m
on sub_e.usdot = m.census_num
left join fleetseek.fleet_primary as fp
on f.fleet_id = fp.fleet_id
left join (
    select 
        fp.fleet_id,
        l1.address1 as phy_address1,
        l1.address2 as phy_address2,
        l1.address3 as phy_address3,
        l1.state_code as phy_state,
        l1.city as phy_city,
        l1.country_code as phy_country,
        l1.zip_code as phy_zip,
        l1.county_fips as phy_countyfips,
        l1.latitude as phy_latitude,
        l1.longitude as phy_longitude,
        l1.location_id as phy_location_id,
        l2.address1 as mai_address1,
        l2.address2 as mai_address2,
        l2.address3 as mai_address3,
        l2.state_code as mai_state,
        l2.city as mai_city,
        l2.country_code as mai_country,
        l2.zip_code as mai_zip,
        l2.county_fips as mai_countyfips,
        l2.latitude as mai_latitude,
        l2.longitude as mai_longitude,
        l2.location_id as mai_location_id

    from fleetseek.location as l1
    join fleetseek.fleet_primary as fp
    on l1.location_id = fp.primary_physical_location_id
    join fleetseek.location as l2
    on fp.primary_mailing_location_id = l2.location_id

) as li
on f.fleet_id = li.fleet_id
left join fleetseek.contact as c
on fp.primary_contact_id = c.contact_id
left join (
    select 
        fleet_id, 
        string_agg(commodity_id::varchar, ', ') as commodities
    from fleetseek.fleet_commodity
    group by fleet_id
) as sub_a
on f.fleet_id = sub_a.fleet_id
left join (
    select 
        pn.phone_number as phone_number, 
        pn.phone_number_id as phone_number_id, 
        pn2.phone_number as fax_number, 
        pn2.phone_number_id as fax_number_id,
        pn3.phone_number as tf_number, 
        pn3.phone_number_id as tf_number_id,
        fp.fleet_id
    from fleetseek.fleet_primary as fp
    left join fleetseek.phone_number as pn
    on fp.primary_phone_id = pn.phone_number_id 
    left join fleetseek.phone_number as pn2
    on fp.primary_fax_id = pn2.phone_number_id
    left join fleetseek.phone_number as pn3
    on fp.primary_toll_free_id = pn3.phone_number_id
) as sub_c
on f.fleet_id = sub_c.fleet_id
left join fleetseek.updates_bn_coder_input as ubci
on f.fleet_id = ubci.fs_fleet_id
left join(
    select 
        min(c.contact_id), 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id
    from fleetseek.fleet_contact as fc
    join fleetseek.fleet as f
    on fc.fleet_id = f.fleet_id
    join fleetseek.contact as c
    on fc.contact_id = c.contact_id
    where 
        sort_order = 2 and
        c.active = true 
    group by 
        fc.fleet_id, 
        sort_order, 
        c.first_name, 
        c.middle_name, 
        c.last_name, 
        c.suffix, 
        c.original_job_title, 
        c.contact_id	
) as sub_b
on fp.fleet_id = sub_b.fleet_id
left join fleetseek.email as e
on fp.primary_email_id = e.email_id
left join (
    select 
        fleet_id, 
        string_agg(trailer_type_id::varchar, ', ') as trailer_type_id
    from fleetseek.fleet_trailer_type
    group by fleet_id
) as sub_d
on f.fleet_id = sub_d.fleet_id;


""")

## Output query

In [50]:
password = 'Evergreen1'

conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
                       host = 'localhost', port = 5433)
cur = conn.cursor()

In [52]:
# Uses string_agg to combine all update reasons if a company appears multiple times

dt_string = datetime.now().strftime("%Y_%m_%d %H_%M_%S")

query = """

select
    mcmis_commodities,
    mcmis_icc1,
    mcmis_act_stat,
    mcmis_carship,
    mcmis_census_num,
    mcmis_dbnum,
    mcmis_rating,
    mcmis_chngdate,
    mcmis_mcs150_date,
    mcmis_class,
    mcmis_classdef,
    mcmis_name,
    mcmis_name_dba,
    mcmis_crrinter,
    mcmis_crrhmintra,
    mcmis_crrintra,
    mcmis_shpinter,
    mcmis_shpintra,
    mcmis_org,
    mcmis_genfreight,
    mcmis_household,
    mcmis_metalsheet,
    mcmis_motorveh,
    mcmis_drivetow,
    mcmis_logpole,
    mcmis_bldgmat,
    mcmis_mobilehome,
    mcmis_machlrg,
    mcmis_produce,
    mcmis_oilfield,
    mcmis_liqgas,
    mcmis_intermodal,
    mcmis_passengers,
    mcmis_livestock,
    mcmis_grainfeed,
    mcmis_coalcoke,
    mcmis_meat,
    mcmis_garbage,
    mcmis_usmail,
    mcmis_chem,
    mcmis_drybulk,
    mcmis_coldfood,
    mcmis_beverages,
    mcmis_paperprod,
    mcmis_utility,
    mcmis_farmsupp,
    mcmis_construct,
    mcmis_waterwell,
    mcmis_cargoothr,
    mcmis_othercargo,
    mcmis_hm_ind,
    mcmis_owntruck,
    mcmis_owntract,
    mcmis_owntrail,
    mcmis_trmtruck,
    mcmis_trmtract,
    mcmis_trmtrail,
    mcmis_trptruck,
    mcmis_trptract,
    mcmis_trptrail,
    mcmis_tot_trucks,
    mcmis_tot_buses,
    mcmis_tot_pwr,
    mcmis_phy_str,
    business_coder_phy_str,
    mcmis_phy_city,
    business_coder_phy_city,
    mcmis_phy_state_code,
    business_coder_phy_state,
    mcmis_phy_zip_code,
    business_coder_phy_zip,
    mcmis_phy_natn,
    mcmis_phy_county,
    business_coder_phy_county,
    business_coder_phy_latitude,
    business_coder_phy_longitude,
    mcmis_undeliv_phy,
    mcmis_mai_str,
    business_coder_mai_str,
    mcmis_mai_city,
    business_coder_mai_city,
    mcmis_mai_state_code,
    business_coder_mai_state,
    mcmis_mai_zip_code,
    business_coder_mai_zip,
    mcmis_mai_natn,
    mcmis_mai_county,
    business_coder_mai_county,
    business_coder_mai_latitude,
    business_coder_mai_longitude,
    mcmis_undeliv_mai,
    mcmis_phone,
    mcmis_fax,
    mcmis_email_address,
    fs_average_miles,
    mcmis_tot_drs,
    mcmis_cdl_drs,
    mcmis_mcs150mileageyear,
    mcmis_mlg150,
    mcmis_ratedate,
    mcmis_adddate,
    mcmis_acc_rate,
    mcmis_company_rep1,
    mcmis_company_rep2,
    upo.fs_fleet_id,
    fs_usdot,
    fs_mcn,
    fs_sic_code,
    fs_update_date,
    fs_active_flag,
    fs_commodities,
    fs_company_type_code,
    fs_product_code,
    fs_company_name,
    fs_phy_address1,
    fs_phy_address2,
    fs_phy_address3,
    fs_phy_state_code,
    fs_phy_city,
    fs_phy_country_code,
    fs_phy_zip_code,
    fs_phy_county_fips,
    fs_phy_latitude,
    fs_phy_longitude,
    fs_physical_location_id,
    fs_mai_address1,
    fs_mai_address2,
    fs_mai_address3,
    fs_mai_state_code,
    fs_mai_city,
    fs_mai_country_code,
    fs_mai_zip_code,
    fs_mai_county_fips,
    fs_mai_latitude,
    fs_mai_longitude,
    fs_mailing_location_id,
    fs_first_name,
    fs_middle_name,
    fs_last_name,
    fs_suffix,
    fs_original_job_title,
    fs_contact1_id,
    fs_first_name2,
    fs_middle_name2,
    fs_last_name2,
    fs_suffix_2,
    fs_original_job_title2,
    fs_contact2_id,
    fs_phone_number,
    fs_phone_id,
    fs_tf_number,
    fs_toll_free_id,
    fs_fax_number,
    fs_fax_id,
    fs_email_address,
    fs_email_id,
    fs_trucks_owned,
    fs_trucks_leased,
    fs_haz_trucks_owned,
    fs_haz_trucks_leased,
    fs_tractors_owned,
    fs_tractors_leased,
    fs_total_vehicles,
    fs_trailers_owned,
    fs_trailers_leased,
    fs_haz_trailers_owned,
    fs_haz_trailers_leased,
    fs_trailer_types,
    fs_tot_drivers,
    fs_cdl_drivers,
    fs_gvw_class_12,
    fs_gvw_class_345,
    fs_gvw_class_6,
    fs_gvw_class_7,
    fs_gvw_class_8,
    fs_revenue,
    fs_website,
    sub_q.update_reason,
    note

from fleetseek.update_output as upo
join (
    select 
        sub_a.fs_fleet_id, 
        string_agg(sub_a.update_reason, ', ') as update_reason
    from (
        select 
            distinct update_reason,
            fs_fleet_id

        from fleetseek.update_output
        group by fs_fleet_id, update_reason

        ) as sub_a
    group by fs_fleet_id
)as sub_q
on upo.fs_fleet_id = sub_q.fs_fleet_id
group by
    mcmis_mlg150,
    upo.mcmis_mcs150mileageyear,
    upo.mcmis_tot_drs,
    upo.mcmis_cdl_drs,
    upo.fs_average_miles,
    mcmis_commodities,
    mcmis_icc1,
    mcmis_act_stat,
    mcmis_carship,
    mcmis_census_num,
    mcmis_dbnum,
    mcmis_rating,
    mcmis_intermodal,
    mcmis_chngdate,
    mcmis_mcs150_date,
    mcmis_class,
    mcmis_classdef,
    mcmis_name,
    mcmis_name_dba,
    mcmis_crrinter,
    mcmis_crrhmintra,
    mcmis_crrintra,
    mcmis_shpinter,
    mcmis_shpintra,
    mcmis_org,
    mcmis_genfreight,
    mcmis_household,
    mcmis_metalsheet,
    mcmis_motorveh,
    mcmis_drivetow,
    mcmis_logpole,
    mcmis_bldgmat,
    mcmis_mobilehome,
    mcmis_machlrg,
    mcmis_produce,
    mcmis_oilfield,
    mcmis_liqgas,
    mcmis_passengers,
    mcmis_livestock,
    mcmis_grainfeed,
    mcmis_coalcoke,
    mcmis_meat,
    mcmis_garbage,
    mcmis_usmail,
    mcmis_chem,
    mcmis_drybulk,
    mcmis_coldfood,
    mcmis_beverages,
    mcmis_paperprod,
    mcmis_utility,
    mcmis_farmsupp,
    mcmis_construct,
    mcmis_waterwell,
    mcmis_cargoothr,
    mcmis_othercargo,
    mcmis_hm_ind,
    mcmis_owntruck,
    mcmis_owntract,
    mcmis_owntrail,
    mcmis_trmtruck,
    mcmis_trmtract,
    mcmis_trmtrail,
    mcmis_trptruck,
    mcmis_trptract,
    mcmis_trptrail,
    mcmis_tot_trucks,
    mcmis_tot_buses,
    mcmis_tot_pwr,
    mcmis_phy_str,
    business_coder_phy_str,
    mcmis_phy_city,
    business_coder_phy_city,
    mcmis_phy_state_code,
    business_coder_phy_state,
    mcmis_phy_zip_code,
    business_coder_phy_zip,
    mcmis_phy_natn,
    mcmis_phy_county,
    business_coder_phy_county,
    business_coder_phy_latitude,
    business_coder_phy_longitude,
    mcmis_undeliv_phy,
    mcmis_mai_str,
    business_coder_mai_str,
    mcmis_mai_city,
    business_coder_mai_city,
    mcmis_mai_state_code,
    business_coder_mai_state,
    mcmis_mai_zip_code,
    business_coder_mai_zip,
    mcmis_mai_natn,
    mcmis_mai_county,
    business_coder_mai_county,
    business_coder_mai_latitude,
    business_coder_mai_longitude,
    mcmis_undeliv_mai,
    mcmis_phone,
    mcmis_fax,
    mcmis_email_address,
    mcmis_ratedate,
    mcmis_adddate,
    mcmis_acc_rate,
    mcmis_company_rep1,
    mcmis_company_rep2,
    upo.fs_fleet_id,
    fs_usdot,
    fs_mcn,
    fs_sic_code,
    fs_update_date,
    fs_active_flag,
    fs_commodities,
    fs_company_type_code,
    fs_product_code,
    fs_company_name,
    fs_phy_address1,
    fs_phy_address2,
    fs_phy_address3,
    fs_phy_state_code,
    fs_phy_city,
    fs_phy_country_code,
    fs_phy_zip_code,
    fs_phy_county_fips,
    fs_phy_latitude,
    fs_phy_longitude,
    fs_physical_location_id,
    fs_mai_address1,
    fs_mai_address2,
    fs_mai_address3,
    fs_mai_state_code,
    fs_mai_city,
    fs_mai_country_code,
    fs_mai_zip_code,
    fs_mai_county_fips,
    fs_mai_latitude,
    fs_mai_longitude,
    fs_mailing_location_id,
    fs_first_name,
    fs_middle_name,
    fs_last_name,
    fs_suffix,
    fs_original_job_title,
    fs_contact1_id,
    fs_first_name2,
    fs_middle_name2,
    fs_last_name2,
    fs_suffix_2,
    fs_original_job_title2,
    fs_contact2_id,
    fs_phone_number,
    fs_phone_id,
    fs_tf_number,
    fs_toll_free_id,
    fs_fax_number,
    fs_fax_id,
    fs_email_address,
    fs_email_id,
    fs_trucks_owned,
    fs_trucks_leased,
    fs_haz_trucks_owned,
    fs_haz_trucks_leased,
    fs_tractors_owned,
    fs_tractors_leased,
    fs_total_vehicles,
    fs_trailers_owned,
    fs_trailers_leased,
    fs_haz_trailers_owned,
    fs_haz_trailers_leased,
    fs_trailer_types,
    fs_tot_drivers,
    fs_cdl_drivers,
    fs_gvw_class_12,
    fs_gvw_class_345,
    fs_gvw_class_6,
    fs_gvw_class_7,
    fs_gvw_class_8,
    fs_revenue,
    fs_website,
    sub_q.update_reason,
    note
order by mcmis_census_num
"""

outputquery = "COPY ({0}) TO STDOUT WITH CSV HEADER".format(query)


with open(r'\\informaplc.sharepoint.com@SSL\DavWWWRoot\sites\GAATLFS1\Shared Documents\FleetSeek Atlanta\ETL_updates\Update_output\Update_output ' + dt_string + '.csv', 'w') as f:
    cur.copy_expert(outputquery, f)

    

## Contact output

In [59]:
query = """
select 
    fc.fleet_id, 
    c.contact_id, 
    c.active as active_contact,
    fc.sort_order,
    c.honorific,
    c.first_name, 
    c.middle_name, 
    c.last_name, 
    c.suffix, 
    c.original_job_title, 
    c.job_title_id,
    l.address1 as phy_address,
    l.address2 as phy_address2,
    l.address3 as phy_address3,
    l.city as phy_city,
    l.county_fips as phy_countyfips,
    l.state_code as phy_state_code,
    l.zip_code as phy_zip_code,
    l.county_name as phy_county_name,
    l.location_id as phy_location_id,
    l.active as phy_active,
    l2.address1 as mai_address,
    l2.address2 as mai_address2,
    l2.address3 as mai_address3,
    l2.city as mai_city,
    l2.county_fips as mai_countyfips,
    l2.state_code as mai_state_code,
    l2.zip_code as mai_zip_code,
    l2.county_name as mai_county_name,
    l2.location_id as mai_location_id,
    l2.active as mai_active,
    pn.phone_number,
    pn.phone_number_id,
    pn.phone_type_id,
    pn.active as phone_active,
    e.email_address,
    e.email_id,
    '' as note


from (
    select 
        distinct fs_fleet_id as fs_fleet_id
    from fleetseek.update_output
) as uo
join fleetseek.fleet_contact as fc
on uo.fs_fleet_id = fc.fleet_id
join fleetseek.contact as c
on fc.contact_id = c.contact_id
join fleetseek.location as l
on c.physical_location_id = l.location_id
join fleetseek.location as l2
on c.mailing_location_id = l2.location_id
join fleetseek.contact_phone as cp
on c.contact_id = cp.contact_id
left join fleetseek.phone_number as pn
on cp.phone_number_id = pn.phone_number_id
left join fleetseek.contact_email as ce
on c.contact_id = ce.contact_id
left join fleetseek.email as e
on ce.email_id = e.email_id
where
    c.active is not false
order by fleet_id
"""

outputquery = "COPY ({0}) TO STDOUT WITH CSV HEADER".format(query)


with open(r'\\informaplc.sharepoint.com@SSL\DavWWWRoot\sites\GAATLFS1\Shared Documents\FleetSeek Atlanta\ETL_updates\Update_output\Contact_output' + dt_string + '.csv', 'w') as f:
    cur.copy_expert(outputquery, f)

In [27]:
cur.execute("""
truncate fleetseek.update_output
""")    
conn.commit()
conn.close()

In [44]:
end_time = time.time() - start_time

In [45]:
print(end_time)

160.7789535522461


### As of 1/30/2020 won't need this one, but keeping it just in case

In [ ]:
# # Name differentiation for review for fleets with 50 or more


# #cur.execute("""
# query = """insert into mcmis_review
# select *
# from mcmis as m
# join (select 
#         f.usdot, 
#         m.census_num,
#         f.total_vehicles
#       from fleetseek.fleet as f
#       join mcmis as m
#       on f.usdot = m.census_num 
#       where upper(left(replace(replace(replace(replace(f.company_name, '.', ''), '&', ''), '-', ''), ' ', ''), 3)) != upper(left(replace(replace(replace(m.name, '-', ''), '&', ''), ' ', ''), 3)) and 
#       f.total_vehicles > 49 and 
#       f.active_flag = 'Y'
#       ) as sub_q
# on m.census_num = sub_q.census_num
# on conflict (mcmis_id) do nothing;

# update mcmis_review
# set review_reason = 'Company-Name'
# where review_reason is null; 
# """)